In [12]:
%config IPCompleter.use_jedi = False
# %xmode Verbose
%xmode context
%pdb off
%load_ext viztracer
from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
from pyphocorehelpers.print_helpers import get_now_time_str, get_now_day_str

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_subsession_neuron_differences, debug_print_ratemap, debug_print_spike_counts, debug_plot_2d_binning, print_aligned_columns
from neuropy.utils.debug_helpers import parameter_sweeps, _plot_parameter_sweep, compare_placefields_info
from neuropy.utils.indexing_helpers import union_of_arrays, intersection_of_arrays, find_desired_sort_indicies, paired_incremental_sorting

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException, document_active_variables
from pyphocorehelpers.general_helpers import GeneratedClassDefinitionType, CodeConversion, inspect_callable_arguments

## Pho Programming Helpers:
import inspect
from pyphocorehelpers.general_helpers import inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables, CapturedException
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsResult, TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer, RankOrderResult


# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')


# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, render_colors

global_data_root_parent_path = find_first_extant_path([Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Exception reporting mode: Context
Automatic pdb calling has been turned OFF
The viztracer extension is already loaded. To reload it, use:
  %reload_ext viztracer
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Pipeline

In [13]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='twolong_LR_pf1Dsession_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False
# 
# # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=['pf_computation',
                                            #    'pfdt_computation',
                                                'firing_rate_trends',
                                                # 'pf_dt_sequential_surprise', 
                                            #    'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                # 'position_decoding_two_step', 
                                            #    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                            #     'long_short_inst_spike_rate_groups',
                                            #     'long_short_endcap_analysis',
                                            # 'split_to_directional_laps',
]

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'



## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated. This will fail because constained versions not ran yet.
if was_updated:
    print(f'was_updated: {was_updated}')
    try:
        curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
    except Exception as e:
        ## TODO: catch/log saving error and indicate that it isn't saved.
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')


basedir: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl... 

INFO:com.PhoHale.Spike3D.pipeline:NeuropyPipeline.__setstate__(state="{'pipeline_name': 'kdiba_pipeline', 'session_data_type': 'kdiba', '_stage': <pyphoplacecellanalysis.General.Pipeline.Stages.Display.DisplayPipelineStage object at 0x000001CFD1266FA0>}")
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze1_odd"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze1_odd] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on fil

done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelis

INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze2_any"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze2_any] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze_any"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_ext

	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
saving_mode.shouldSave == False, so not saving at the end of batch_load_session
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricate

In [14]:
### GLOBAL COMPUTATIONS:
if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise

curr_active_pipeline.reload_default_computation_functions()

extended_computations_include_includelist=['pf_computation', 'firing_rate_trends', # 'pfdt_computation',
    # 'pf_dt_sequential_surprise',
    #  'ratemap_peaks_prominence2d',
    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding',
    # 'long_short_rate_remapping',
    # 'long_short_inst_spike_rate_groups',
    'long_short_endcap_analysis',
    # 'spike_burst_detection',
    'split_to_directional_laps',
    'rank_order_shuffle_analysis'
] # do only specified

force_recompute_global = force_reload
# force_recompute_global = True
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0):
    print(f'newly_computed_values: {newly_computed_values}.')
    if (saving_mode.value != 'skip_saving'):
        print(f'Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'\n\n!!WARNING!!: changes to global results have been made but they will not be saved since saving_mode.value == "skip_saving"')
        print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
    print(f'no changes in global results.')

# except Exception as e:
#     exception_info = sys.exc_info()
#     e = CapturedException(e, exception_info)
#     print(f'second half threw: {e}')


# 4m 5.2s for inst fr computations
# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl... done.
included includelist is specified: ['pf_computation', 'firing_rate_trends', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_endcap_analysis', 'split_to_directional_laps', 'rank_order_shuffle_analysis'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze_any"
pf_computation, maze_any already computed.
firing_rate_trends, maze_any already computed.
split_to_directional_laps, maze_any already computed.
rank_order_shuffle_analysis, maze_any already computed.
long_short_decoding_analyses, maze_any already computed.
short_long_pf_overlap_analyses, maze_any already computed.
long_short_fr_indicies_analyses, maze_any already computed.
jonathan_firing_rate_analysis, maze_a

In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
curr_active_pipeline.export_pipeline_to_h5()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE)

# End Run

In [15]:
# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name.removesuffix('_any'))) for an_epoch_name in [long_epoch_name, short_epoch_name]] #TODO 2023-11-10 20:41: - [ ] Issue with getting actual Epochs from sess.epochs for directional laps: emerges because long_epoch_name: 'maze1_any' and the actual epoch label in curr_active_pipeline.sess.epochs is 'maze1' without the '_any' part.
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D

assert short_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'
assert long_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'

In [16]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global 
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global `long_short_fr_indicies_analysis`:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]
long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.05)

## Update long_exclusive/short_exclusive properties with `long_short_fr_indicies_df`
# long_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)
# short_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)


WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above
WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above


In [17]:
# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']

# Most popular
# long_LR_name, short_LR_name, long_RL_name, short_RL_name, global_any_name

# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)


In [18]:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: float = rank_order_results.included_qclu_values
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')

minimum_inclusion_fr_Hz: 5.0
included_qclu_values: [1, 2]


In [19]:
# NEW 2023-11-22 method: Get the templates (which can be filtered by frate first) and the from those get the decoders):        
# track_templates: TrackTemplates = directional_laps_results.get_shared_aclus_only_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # shared-only
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']
# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)

LR_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

LR_results_long_short_z_diffs = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_long_short_z_diff = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

In [ ]:
active_burst_intervals = curr_active_pipeline.computation_results[global_epoch_name].computed_data['burst_detection']['burst_intervals']
# active_burst_intervals

In [ ]:
# Relative Entropy/Surprise Results:
active_extended_stats = global_results['extended_stats']
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
historical_snapshots = active_relative_entropy_results['historical_snapshots']
post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
surprise_time_bin_duration = (post_update_times[2]-post_update_times[1])
long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)

## Get the placefield dt matrix:
if 'snapshot_occupancy_weighted_tuning_maps' not in active_relative_entropy_results:
	## Compute it if missing:
	occupancy_weighted_tuning_maps_over_time = np.stack([placefield_snapshot.occupancy_weighted_tuning_maps_matrix for placefield_snapshot in historical_snapshots.values()])
	active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] = occupancy_weighted_tuning_maps_over_time
else:
	occupancy_weighted_tuning_maps_over_time = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] # (n_post_update_times, n_neurons, n_xbins)


In [ ]:
# Time-dependent
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt

In [ ]:
## long_short_endcap_analysis: checks for cells localized to the endcaps that have their placefields truncated after shortening the track
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
# disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
# trivially_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
# significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index
# appearing_aclus

In [ ]:
epoch_active_aclus = np.array([9,  26,  31,  39,  40,  43,  47,  52,  53,  54,  60,  61,  65,  68,  72,  75,  77,  78,  81,  82,  84,  85,  90,  92,  93,  98, 102])


curr_active_pipeline.filtered_contexts['maze1_odd']
curr_active_pipeline.computation_results['maze1_odd']

In [ ]:
curr_active_pipeline.prepare_for_display()
curr_active_pipeline.display('_display_1d_placefields', 'maze1_odd') # , 'maze1_odd'


In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', write_vector_format=True, write_png=True, debug_print=True)

In [ ]:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.prepare_for_display()

In [ ]:

_out = curr_active_pipeline.display('_display_1d_placefields', 'maze_any')


In [ ]:
_out

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze_any')

In [ ]:
curr_active_pipeline.display('_display_placemaps_pyqtplot_2D', 'maze2_odd')

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_odd')

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', 'maze1_odd')

In [ ]:

# Adjust layout to make space for the footer
# plt.subplots_adjust(bottom=0.35)

plt.tight_layout(pad=2.0)

In [ ]:
_display_placemaps_pyqtplot_2D

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
from mpl_multitab import MplMultiTab, MplMultiTab2D
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_display_to_PDF, programmatic_render_to_file
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_single_cell_1D_placecell_validation
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1d_placecell_validations


# matplotlib_configuration_update(is_interactive=True)

# curr_active_pipeline.display('_display_grid_bin_bounds_validation')
_out = curr_active_pipeline.display('_display_1d_placefield_validations', 'maze1_odd')
_out.ui.show()

In [ ]:

programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations', filter_name='maze1_odd', debug_print=True)

# plt.show()

In [ ]:
placefield_cell_index = 0
active_epoch_placefields1D = deepcopy(long_pf1D)
curr_cell_normalized_tuning_curve = active_epoch_placefields1D.ratemap.normalized_tuning_curves[placefield_cell_index, :].squeeze()
{'xbin_centers': active_epoch_placefields1D.ratemap.xbin_centers, 'curr_cell_normalized_tuning_curve': curr_cell_normalized_tuning_curve}

{'xbin_centers': np.array([31.0565, 34.8495, 38.6426, 42.4356, 46.2286, 50.0216, 53.8147, 57.6077, 61.4007, 65.1937, 68.9867, 72.7798, 76.5728, 80.3658, 84.1588, 87.9519, 91.7449, 95.5379, 99.3309, 103.124, 106.917, 110.71, 114.503, 118.296, 122.089, 125.882, 129.675, 133.468, 137.261, 141.054, 144.847, 148.64, 152.433, 156.226, 160.019, 163.812, 167.605, 171.398, 175.191, 178.984, 182.777, 186.57, 190.363, 194.157, 197.95, 201.743, 205.536, 209.329, 213.122, 216.915, 220.708, 224.501, 228.294, 232.087, 235.88, 239.673, 243.466, 247.259, 251.052, 254.845, 258.638, 262.431]),
 'curr_cell_normalized_tuning_curve': np.array([5.92979e-05, 0.000150933, 0.00036895, 0.000736517, 0.00121915, 0.00173714, 0.0022042, 0.00252859, 0.0026496, 0.0027108, 0.00312627, 0.00423033, 0.00579314, 0.00709557, 0.00766535, 0.00789647, 0.00884807, 0.0115452, 0.0165549, 0.0238423, 0.0323681, 0.039895, 0.0442459, 0.0452642, 0.0449909, 0.0457691, 0.0485138, 0.0525281, 0.0562324, 0.0581433, 0.0575758, 0.0544383, 0.0486438, 0.0404683, 0.0315115, 0.0243731, 0.0207242, 0.0199181, 0.0197507, 0.0183449, 0.0153819, 0.0119837, 0.00951012, 0.00827676, 0.00740415, 0.00596512, 0.00396809, 0.00210018, 0.000875453, 0.000302685, 0.000153468, 0.00027615, 0.000667689, 0.00135676, 0.00224608, 0.00305331, 0.0034339, 0.0031979, 0.0024518, 0.00153458, 0.00079294, 0.000405152])}


In [ ]:
# batch_extended_programmatic_figures


In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze1_odd')

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_even')

In [ ]:

#TODO 2023-11-29 09:18: - [ ] Not good, the self.filtered_contexts are not unique!
list(curr_active_pipeline.filtered_contexts.values())
# [IdentifyingContext<(... 'maze2')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>, IdentifyingContext<(...ze1_any')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>]
[(v == curr_active_pipeline.filtered_contexts['maze1_even']) for v in list(curr_active_pipeline.filtered_contexts.values())]
# [True, True, False, True, True, False, False, True, False]
# meaning `curr_active_pipeline.display('_display_1d_placefields', curr_active_pipeline.filtered_contexts['maze1_even'])` doesn't work
curr_active_pipeline.filtered_contexts.index(curr_active_pipeline.filtered_contexts['maze1_even'])

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_odd')


In [ ]:
write_vector_format = False
write_png = True
debug_print = True
from neuropy.plotting.ratemaps import BackgroundRenderingOptions

programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D', write_vector_format=write_vector_format, write_png=write_png, debug_print=debug_print, bg_rendering_mode=BackgroundRenderingOptions.EMPTY) #  🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.


In [ ]:
_out = curr_active_pipeline.display('_display_2d_placefield_occupancy', 'maze2_any')

In [ ]:
_out = curr_active_pipeline.display('_display_2d_placefield_occupancy', 'maze1_any')
occupancy_ax = _out.axes #.get_aspect()
pf = long_pf2D
# pf.xbin
# pf.ybin
# pf.xbin_centers
# pf.ybin_centers

# aspect_ratio = np.ptp(pf.xbin) / np.ptp(pf.ybin)  # ptp: peak to peak (range)
# aspect_ratio = 0.102803738317757
# print(f'aspect_ratio: {aspect_ratio}')
# occupancy_ax.set_aspect(aspect_ratio, adjustable='box') # If 'box', change the physical dimensions of the Axes. If 'datalim', change the x or y data limits.


## See 
# https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_anchor.html#


occupancy_ax.set_aspect('equal', adjustable=None)


In [ ]:
occupancy_ax.set_aspect('equal', adjustable='datalim')

In [ ]:
occupancy_ax.set_aspect('equal', adjustable='box')

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_occupancy', write_vector_format=write_vector_format, write_png=write_png, debug_print=debug_print)

# EVEN: "RL", ODD: "LR"
Starts with Even (idx=0)
- EVEN: "RL"
shared_RL_aclus_only_neuron_IDs
`is_even = (an_epoch.lap_dir == 0)`
- ODD: "LR"
shared_LR_aclus_only_neuron_IDs
`is_odd = (an_epoch.lap_dir == 1)`

# 🟢 2023-10-20 - Z-Score Comparisons with Neuron_ID Shuffled templates
1. Take the intersection of the long and short templates to get only the common cells
2. Determine the long and short "tempaltes": this is done by ranking the aclus for each by their placefields' center of mass. `compute_placefield_center_of_masses`
	2a. `long_pf_peak_ranks`, `short_pf_peak_ranks` - there are one of each of these for each shared aclu.
3. Generate the unit_id shuffled (`shuffled_aclus`, `shuffle_IDXs`) ahead of time to use to shuffle the two templates during the epochs.
4. For each replay event, take each shuffled template
	4a. Iterate through each shuffle and obtain the shuffled templates like `long_pf_peak_ranks[epoch_specific_shuffled_indicies]`, `short_pf_peak_ranks[epoch_specific_shuffled_indicies]`
	4b. compute the spearman rank-order of the event and each shuffled template, and accumulate the results in `long_spearmanr_rank_stats_results`, `short_spearmanr_rank_stats_results`

5. After we're done with the shuffle loop, accumulate the results and convert to the right output format.

6. When all epochs are done, loop through the results (the epochs again) and compute the z-scores for each epoch so they can be compared to each other. Keep track of the means and std_dev for comparisons later, and subtract the two sets of z-scores (long/short) to get the delta_Z for each template.

7. TODO: Next figure out what to do with the array of z-scores and delta_Z. We have:
	n_epochs sets of results
		n_shuffles scores of delta_Z



## Convo with Kamran 2023-10-23:
- Use directional templates **
- No need to worry about re-ranking
[X] Plot the long and short separately in addition to the difference, so we show significant reqplay on each as a sanity check
[X] Absolute value difference?
[X] Fisher transform the correlation values (check if there is a difference) because correlation coefficients aren't going to be normally distributed.
	[ ] Then Z-score releative to fisher.

- T-test to compare to mean of zero (if looking at the difference)

In [ ]:
## Concerns:
# 1. Permutation recommended over shuffling for small numbers of ids
# 2.

# 5Hz thresholding of templates


In [20]:
from nptyping import NDArray
from attrs import define, field, Factory, astuple
import scipy.stats
from scipy import ndimage
from neuropy.utils.misc import build_shuffled_ids # used in _SHELL_analyze_leave_one_out_decoding_results
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_paired_t_test


# minimum_inclusion_fr_Hz: float = 2.0
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz

# Recover from the saved global result:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
# NEW 2023-11-22 method: Get the templates (which can be filtered by frate first) and the from those get the decoders):        
# track_templates: TrackTemplates = directional_laps_results.get_shared_aclus_only_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # shared-only
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

## Pre 2023-11-22 method: building a TrackTemplates object after getting the raw decoders:
# long_LR_one_step_decoder_1D, long_RL_one_step_decoder_1D, short_LR_one_step_decoder_1D, short_RL_one_step_decoder_1D = directional_laps_results.get_decoders()
# long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = directional_laps_results.get_shared_aclus_only_decoders()
# track_templates: TrackTemplates = TrackTemplates.init_from_paired_decoders(LR_decoder_pair=(long_LR_decoder, short_LR_decoder), RL_decoder_pair=(long_RL_decoder, short_RL_decoder))
# # track_templates: TrackTemplates = TrackTemplates.init_from_paired_decoders(LR_decoder_pair=(long_LR_one_step_decoder_1D, short_LR_one_step_decoder_1D), RL_decoder_pair=(long_RL_one_step_decoder_1D, short_RL_one_step_decoder_1D)) # NOTE: now use the un-constrained versions

# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']
# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)

LR_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

LR_results_long_short_z_diffs = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_long_short_z_diff = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])


# LR_results_real_values

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import plot_rank_order_histograms

# Plot histograms:
post_title_info: str = f'{minimum_inclusion_fr_Hz} Hz\n{curr_active_pipeline.get_session_context().get_description()}'
_out_z_score, _out_real, _out_most_likely_z = plot_rank_order_histograms(rank_order_results, post_title_info=post_title_info)


#TODO 2023-12-10 19:56: - [ ] Histogram Display Helpers

#TODO 2023-12-10 19:56: - [ ] Pf1D Helpers

#TODO 2023-12-10 19:57: - [ ] Variant Saving
 

In [ ]:
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=0.0) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

# filtered_decoder_list = [filtered_by_frate(a_decoder, minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, debug_print=True) for a_decoder in (long_LR_one_step_decoder_1D, long_RL_one_step_decoder_1D, short_LR_one_step_decoder_1D, short_RL_one_step_decoder_1D)]
original_neuron_ids_list = [a_decoder.pf.ratemap.neuron_ids for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]
is_aclu_included_list = [a_decoder.pf.ratemap.tuning_curve_unsmoothed_peak_firing_rates >= minimum_inclusion_fr_Hz for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]
filtered_aclus_list = [np.array(a_decoder.pf.ratemap.neuron_ids)[a_decoder.pf.ratemap.tuning_curve_unsmoothed_peak_firing_rates >= minimum_inclusion_fr_Hz] for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]

## For a given run direction (LR/RL) let's require inclusion in either (OR) long v. short to be included.
filtered_included_LR_aclus = np.union1d(filtered_aclus_list[0], filtered_aclus_list[2])
filtered_included_RL_aclus = np.union1d(filtered_aclus_list[1], filtered_aclus_list[3])
# build the final shared aclus:
filtered_direction_shared_aclus_list = [filtered_included_LR_aclus, filtered_included_RL_aclus, filtered_included_LR_aclus, filtered_included_RL_aclus] # contains the shared aclus for that direction
# rebuild the is_aclu_included_list from the shared aclus
is_aclu_included_list = [np.isin(an_original_neuron_ids, a_filtered_neuron_ids) for an_original_neuron_ids, a_filtered_neuron_ids in zip(original_neuron_ids_list, filtered_direction_shared_aclus_list)]

# is_aclu_included_list[0]
filtered_direction_shared_aclus_list

In [ ]:
# # for 5Hz:
# [array([  5,   7,  31,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  95,  99, 100, 108]),
#  array([  5,   7,   9,  31,  32,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  93,  95,  99, 101, 108]),
#  array([  5,   7,  31,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  95,  99, 100, 108]),
#  array([  5,   7,   9,  31,  32,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  93,  95,  99, 101, 108])]

# # for 20Hz:
# [array([  5,  41,  46,  48,  69,  78,  79,  83,  86,  88,  90, 108]),
#  array([ 62,  64,  75,  78,  83,  91, 101]),
#  array([  5,  41,  46,  48,  69,  78,  79,  83,  86,  88,  90, 108]),
#  array([ 62,  64,  75,  78,  83,  91, 101])]

# 2023-11-22 - RECOMPUTE

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalLapsHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions

curr_active_pipeline.reload_default_computation_functions()

## clear the old values to prepare for the new ones:
curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'] = None
curr_active_pipeline.global_computation_results.computed_data['RankOrder'] = None
del curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
del curr_active_pipeline.global_computation_results.computed_data['RankOrder']


curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'] = DirectionalLapsHelpers.build_global_directional_result_from_natural_epochs(curr_active_pipeline, progress_print=True) # repalce the directional laps object
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']

minimum_inclusion_fr_Hz = 5.0
included_qclu_values = [1,2]

# minimum_inclusion_fr_Hz = 1.0
# included_qclu_values = [1,2,4,9]

# perform_rank_order_shuffle_analysis
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-perform_rank_order_shuffle_analysis_{curr_active_pipeline.session_name}.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
RankOrderGlobalComputationFunctions.perform_rank_order_shuffle_analysis(curr_active_pipeline, curr_active_pipeline.global_computation_results, None, None, include_includelist=None, debug_print=True,
                                                                        num_shuffles=750, minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values, skip_laps=False)


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
selected_spikes_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.selected_spikes_df)
# active_epochs = global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple.active_epochs
active_epochs = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.epochs_df)
track_templates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz)
ripple_combined_epoch_stats_df, ripple_new_output_tuple = RankOrderAnalyses.pandas_df_based_correlation_computations(selected_spikes_df=selected_spikes_df, active_epochs_df=active_epochs, track_templates=track_templates, num_shuffles=5)

In [ ]:
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: List[int] = rank_order_results.included_qclu_values
ripple_result_tuple, laps_result_tuple = rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only -- !! Is minimum_inclusion_fr_Hz=None the issue/difference?
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')

# 10m 29.5s for 1000 shuffles.  c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\viztracer_2023-11-22_16-11-perform_rank_order_shuffle_analysis.json

# 3m 33.9s - 500
# 3m 26.4s - 1000

In [ ]:
_ripples_outputs = RankOrderAnalyses.main_ripples_analysis(curr_active_pipeline, num_shuffles=500, rank_alignment='median', minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values)
(LR_ripple_outputs, RL_ripple_outputs, ripple_evts_paired_tests) = _ripples_outputs


In [ ]:
# LR_ripple_outputs.epochs_df
LR_ripple_outputs.spikes_df

In [ ]:
RL_ripple_outputs.spikes_df

In [ ]:
LR_ripple_outputs.selected_spikes_df

In [ ]:
RL_ripple_outputs.



In [ ]:
## Ensure equivalence of the two LR_ripple_outputs and RL_ripple_outputs for the fields that matter:
assert LR_ripple_outputs.spikes_df.equals(RL_ripple_outputs.spikes_df), f"spikes_df are not equal"
assert LR_ripple_outputs.selected_spikes_df.equals(RL_ripple_outputs.selected_spikes_df), f"selected_spikes_df are not equal"
assert LR_ripple_outputs.epochs_df.equals(RL_ripple_outputs.epochs_df), f"epochs_df are not equal"

In [ ]:
_new_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(deepcopy(LR_ripple_outputs.selected_spikes_df), deepcopy(LR_ripple_outputs.epochs_df), track_templates, rank_order_results, None, None)



In [ ]:
# TypeError: <lambda>() missing 1 required positional argument


In [ ]:
## Recompute just the `most_likely_directional_rank_order_shuffling` part:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderLikelihoods
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses
## Main
ripple_result_tuple, laps_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.003)
rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple = ripple_result_tuple, laps_result_tuple

In [ ]:
directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = deepcopy(ripple_result_tuple.directional_likelihoods_tuple)
directional_likelihoods_tuple.long_best_direction_indices
directional_likelihoods_tuple.short_best_direction_indices
# directional_likelihoods_tuple.long_relative_direction_likelihoods





# Saving/Loading `DirectionalLaps_2Hz`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import save_rank_order_results

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData

# Load the data from a file into the pipeline:
# out_filename_str: str = '2023-12-11-minimum_inclusion_fr_Hz_2_included_qclu_values_1-2_' # specific

minimum_inclusion_fr_Hz: float = 1.0
included_qclu_values: List[int] = [1,2,4,9]
out_filename_str = generate_save_suffix(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values, day_date='2023-12-19_530pm')
print(f'out_filename_str: "{out_filename_str}"')
# day_date_str: str = '2023-12-11_with_tuple_newer_'
# day_date_str: str = ''
directional_laps_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}DirectionalLaps.pkl').resolve()
assert directional_laps_output_path.exists()
# loaded_directional_laps, loaded_rank_order = loadData(directional_laps_output_path)
loaded_directional_laps = loadData(directional_laps_output_path)
assert (loaded_directional_laps is not None)
# assert (loaded_rank_order is not None)
# Apply the loaded data to the pipeline:
# curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'], curr_active_pipeline.global_computation_results.computed_data['RankOrder'] = loaded_directional_laps, loaded_rank_order
# curr_active_pipeline.global_computation_results.computed_data['RankOrder']


In [ ]:
rank_order_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}RankOrder.pkl').resolve()
loaded_rank_order = loadData(rank_order_output_path)
loaded_rank_order
# asdict(loaded_directional_laps, recurse=True) == asdict(curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'], recurse=True)

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

In [ ]:
rank_order_results.LR_ripple.selected_spikes_df

# POST-Compute:

In [21]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalDisplayFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multi_sort_raster_browser
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import RankOrderRastersDebugger

from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import paired_separately_sort_neurons, paired_incremental_sort_neurons # _display_directional_template_debugger
from neuropy.utils.indexing_helpers import paired_incremental_sorting, union_of_arrays, intersection_of_arrays, find_desired_sort_indicies
from pyphoplacecellanalysis.GUI.Qt.Widgets.ScrollBarWithSpinBox.ScrollBarWithSpinBox import ScrollBarWithSpinBox

from neuropy.utils.mixins.HDF5_representable import HDF_SerializationMixin
from pyphoplacecellanalysis.General.Model.ComputationResults import ComputedResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses, RankOrderResult, ShuffleHelper, Zscorer, LongShortStatsTuple, DirectionalRankOrderLikelihoods, DirectionalRankOrderResult, RankOrderComputationsContainer
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import TimeColumnAliasesProtocol
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer

## Display Testing
# from pyphoplacecellanalysis.External.pyqtgraph import QtGui
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import pyqtplot_build_image_bounds_extent, pyqtplot_plot_image


spikes_df = curr_active_pipeline.sess.spikes_df
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: List[int] = rank_order_results.included_qclu_values
ripple_result_tuple, laps_result_tuple = rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only -- !! Is minimum_inclusion_fr_Hz=None the issue/difference?
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')
# ripple_result_tuple

## Unpacks `rank_order_results`: 
# global_replays = Epoch(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# active_replay_epochs, active_epochs_df, active_selected_spikes_df = combine_rank_order_results(rank_order_results, global_replays, track_templates=track_templates)
# active_epochs_df

# 2023-12-15 - Newest method:
laps_combined_epoch_stats_df = rank_order_results.laps_combined_epoch_stats_df
ripple_combined_epoch_stats_df = rank_order_results.ripple_combined_epoch_stats_df
ripple_combined_epoch_stats_df


minimum_inclusion_fr_Hz: 5.0
included_qclu_values: [1, 2]


,LR_Long_spearman,RL_Long_spearman,LR_Short_spearman,RL_Short_spearman,LR_Long_pearson,RL_Long_pearson,LR_Short_pearson,RL_Short_pearson,LR_Long_spearman_Z,RL_Long_spearman_Z,LR_Short_spearman_Z,RL_Short_spearman_Z,LR_Long_pearson_Z,RL_Long_pearson_Z,LR_Short_pearson_Z,RL_Short_pearson_Z,label
0,-0.289886,0.600000,-0.142857,0.200000,-0.276699,0.255187,-0.052640,0.270244,-0.700573,1.522588,-0.337940,0.504844,-0.668865,0.646215,-0.121958,0.683831,0.0
1,0.142857,-0.309626,-0.292770,0.470605,-0.003213,-0.373721,-0.350008,0.464134,0.343225,-0.788045,-0.698073,1.190707,-0.009938,-0.950376,-0.835123,1.175891,1.0
2,-0.737865,0.107143,-0.600000,0.666694,-0.601564,-0.312179,-0.287031,0.303629,-1.781122,0.270632,-1.436125,1.687704,-1.451586,-0.794143,-0.684089,0.768555,2.0
3,0.657143,0.500000,0.942857,0.800000,0.701677,0.766788,0.990892,0.619848,1.583709,1.268568,2.270249,2.025576,1.688402,1.945000,2.380699,1.571066,3.0
4,0.445394,0.206612,0.333333,0.234161,0.601733,0.361404,0.443333,0.076471,1.072959,0.523304,0.806003,0.591426,1.447601,0.915864,1.067511,0.192067,4.0
5,0.316228,0.737865,0.816497,1.000000,0.620064,0.801341,0.958722,0.951276,0.761404,1.872792,1.966696,2.532487,1.491767,2.032717,2.303546,2.412172,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.181818,-0.154914,0.542143,0.204993,0.265933,0.384650,0.549011,0.327272,0.437201,-0.395045,1.307621,0.517500,0.638534,0.974878,1.320955,0.828558,274.0
274,-0.380952,-0.226899,-0.365963,-0.085126,-0.407567,-0.290440,-0.378760,-0.131271,-0.920231,-0.577901,-0.873901,-0.217822,-0.984175,-0.738954,-0.904077,-0.335145,275.0
275,-0.229412,-0.619276,-0.441005,-0.519946,-0.029414,-0.728461,-0.248263,-0.630800,-0.554707,-1.574618,-1.054174,-1.319897,-0.073065,-1.850945,-0.591112,-1.602864,NaN


In [ ]:
ripple_result_tuple.active_epochs

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple

In [ ]:
np.logical_not(np.isnan(rank_order_results.ripple_combined_epoch_stats_df.index).any())
# ripple_combined_epoch_stats_df.label.isna()

In [ ]:
ripple_combined_epoch_stats_df

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.label).any()

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.index).any()

In [ ]:
print(f'\tdone. building global result.')
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
selected_spikes_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.selected_spikes_df)
# active_epochs = global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple.active_epochs
active_epochs = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.epochs_df)
track_templates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz)
ripple_combined_epoch_stats_df, ripple_new_output_tuple = RankOrderAnalyses.pandas_df_based_correlation_computations(selected_spikes_df=selected_spikes_df, active_epochs_df=active_epochs, track_templates=track_templates, num_shuffles=150)
# new_output_tuple (output_active_epoch_computed_values, valid_stacked_arrays, real_stacked_arrays, n_valid_shuffles) = ripple_new_output_tuple
curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_combined_epoch_stats_df, curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_new_output_tuple = ripple_combined_epoch_stats_df, ripple_new_output_tuple
print(f'done!')

In [ ]:
laps_combined_epoch_stats_df

In [ ]:
print_object_memory_usage(output_active_epoch_computed_values) # 0.946189 MB


In [ ]:
## #TODO 2023-12-13 02:07: - [ ] Figure out how 'Probe_Epoch_id' maps to `ripple_result_tuple.active_epochs`
ripple_result_tuple.active_epochs
rank_order_results.LR_ripple.ranked_aclus_stats_dict


In [ ]:
## Add the pf_x information for each aclu:
## 2023-10-11 - Get the long/short peak locations
# decoder_peak_coms_list = [a_decoder.pf.ratemap.peak_tuning_curve_center_of_masses[is_good_aclus] for a_decoder in decoder_args]
decoder_aclu_peak_location_dict_list = [dict(zip(neuron_IDs, peak_locations)) for neuron_IDs, peak_locations in zip(track_templates.decoder_neuron_IDs_list, track_templates.decoder_peak_location_list)]
decoder_aclu_peak_location_dict_list


In [ ]:
track_templates.long_LR_decoder.peak_locations

In [ ]:
track_templates.decoder_LR_pf_peak_ranks_list

In [ ]:
## Replays:
global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
if isinstance(global_replays, pd.DataFrame):
	global_replays = Epoch(global_replays.epochs.get_valid_df())

# get the aligned epochs and the z-scores aligned to them:
active_replay_epochs, (active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score) = rank_order_results.get_aligned_events(global_replays.to_dataframe().copy(), is_laps=False)
active_replay_epochs

In [ ]:
## Laps:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
global_laps = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].laps).trimmed_to_non_overlapping()
active_laps_epochs, (active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score) = rank_order_results.get_aligned_events(global_laps.to_dataframe(), is_laps=True)

In [ ]:
ripple_result_tuple.plot_histogram()

In [ ]:
# Find only the significant events (|z| > 1.96):
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

filtered_z_score_df, (n_events, n_significant_events, percent_significant_events) = RankOrderAnalyses.find_only_significant_events(rank_order_results, high_z_criteria=1.96)
filtered_z_score_df

In [ ]:
print(filtered_z_score_df.index.to_numpy())


In [ ]:
# 2023-11-20 - Finding high-significance periods for Kamran:
z_threshold = 1.96
is_greater_than_z_threshold_long = (np.abs(ripple_result_tuple.long_best_dir_z_score_values) > z_threshold)
is_greater_than_z_threshold_short = (np.abs(ripple_result_tuple.short_best_dir_z_score_values) > z_threshold)
is_significant_either = np.logical_or(is_greater_than_z_threshold_long, is_greater_than_z_threshold_short)
is_significant_either

# is_greater_than_3std_long = (np.abs(ripple_result_tuple.long_best_dir_z_score_values) >= 3.0)
# is_greater_than_3std_short = (np.abs(ripple_result_tuple.short_best_dir_z_score_values) >= 3.0)
# is_significant_either = np.logical_or(is_greater_than_3std_long, is_greater_than_3std_short)


In [ ]:
significant_ripple_epochs = deepcopy(Epoch(ripple_result_tuple.active_epochs)).boolean_indicies_slice(is_significant_either)
# significant_ripple_epochs = deepcopy(global_replays).boolean_indicies_slice(is_significant_either)
significant_ripple_epochs.to_dataframe()

# significant_ripple_epochs.filename = Path(f'output/2023-11-27_SignificantReplayRipples').resolve()
# significant_ripple_epochs.to_neuroscope()


In [ ]:
# active_epochs = ripple_result_tuple.active_epochs
active_epochs: Epoch = rank_order_results.RL_ripple.epochs_df # Epoch(rank_order_results.RL_ripple.epochs_df)
# type(active_epochs)
active_epochs.n_epochs
# rank_order_results.RL_ripple.spikes_df

In [ ]:
rank_order_results.LR_ripple.epochs_df
rank_order_results.LR_ripple.spikes_df



In [ ]:
combined_variable_names: ['LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson']
combined_variable_z_score_column_names: ['LR_Long_spearman_Z', 'RL_Long_spearman_Z', 'LR_Short_spearman_Z', 'RL_Short_spearman_Z', 'LR_Long_pearson_Z', 'RL_Long_pearson_Z', 'LR_Short_pearson_Z', 'RL_Short_pearson_Z']

        


In [ ]:
curr_active_pipeline.build_display_context_for_filtered_session(filtered_session_name='maze_any', display_fn_name='test')

In [ ]:
rank_order_results.LR_ripple.selected_spikes_df

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

# `RankOrderRastersDebugger`

In [23]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import RankOrderRastersDebugger
from pyphocorehelpers.gui.Qt.color_helpers import build_adjusted_color
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles

curr_active_pipeline.reload_default_display_functions()

## RankOrderRastersDebugger: 
# _out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, ripple_result_tuple.active_epochs, track_templates, rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)

## Required prereqs:
# active_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
# active_selected_spikes_df, active_epochs_df = RankOrderAnalyses.new_compute_correlations(selected_spikes_df=selected_spikes_df, active_epochs=active_epochs, track_templates=track_templates)
## 2023-12-13 11am - uses `active_epochs_df` passed in that has been augmented with all the old results:
# _out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, active_epochs_df, track_templates, rank_order_results, rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)
# _out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, active_epochs_df, track_templates, rank_order_results, rank_order_results.RL_ripple.selected_spikes_df, rank_order_results.LR_ripple.selected_spikes_df)
# _out_rank_order_event_raster_debugger.params.enable_show_Z_values = False

_out_rank_order_selected_spikes_only_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(deepcopy(rank_order_results.LR_ripple.selected_spikes_df), deepcopy(rank_order_results.LR_ripple.epochs_df), track_templates, rank_order_results, None, None) # rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)


included_neuron_ids: [  2   4   5   6   8   9  11  12  14  15  16  18  19  20  24  25  26  27  28  29  30  31  32  33  34  35  38  39  40  43  44  47  48  51  52  53  56  57  58  59  60  61  62  63  66  67  68  70  71  72  75  77  79  80  81  82  83  84  85  86  87  89  90  91  92  93  95  98 101 102 104], n_neurons: 71
unit_sort_order: [10 22 27 45 46  6 20 41 44  8 15 16 24 26 21 23 36 13 38 33 48 17  4 47 39  0 14  5 31 43  1  3 28  2 12  7 50 37  9 34 40 19 29 42 25 49 11 18 30 32 35]
desired_sort_arr: [ 25  47  56  89  92  15  43  84  87  19  33  34  51  53  44  48  79  30  81  72  98  35   9  93  82   2  31  11  66  86   5   8  58   6  29  16 104  80  24  75  83  40  60  85  52 102  26  39  61  68  77]
WARN: len(neuron_colors): 71 > n_cells: 51: restricting neuron_colors to the correct aclus, but if colors ever get off this is where it is happening!
merged_kwargs: {'name': 'spikeRasterOverviewWindowScatterPlotItem', 'pxMode': True, 'symbol': <PyQt5.QtGui.QPainterPath object at 0x

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, ExportFiletype

export_path = Path(r'C:\Users\pho\Desktop\2023-12-19 Exports').resolve()

_out_rank_order_event_raster_debugger.save_figure(export_path=export_path)


## Iterates through the epochs (via the slider) and saves out the images:


In [ ]:
all_save_paths = {}

for i in np.arange(0, _out_rank_order_event_raster_debugger.n_epochs, 5):
    _out_rank_order_event_raster_debugger.ui.ctrls_widget.setValue(i)
    # _out_rank_order_event_raster_debugger.on_update_epoch_IDX(an_epoch_idx=i)
    active_epoch_label = _out_rank_order_event_raster_debugger.active_epoch_label

    save_paths = []

    for a_decoder, a_plot in _out_rank_order_event_raster_debugger.root_plots_dict.items():
        curr_filename_prefix = f'Epoch{active_epoch_label}_{a_decoder}'
        # a_plot.setYRange(-0.5, float(self.max_n_neurons))
        out_path = export_path.joinpath(f'{curr_filename_prefix}_plot.png').resolve()
        export_pyqtgraph_plot(a_plot, savepath=out_path, background=pg.mkColor(0, 0, 0, 0))
        save_paths.append(out_path)

    all_save_paths[active_epoch_label] = save_paths
  
all_save_paths

In [ ]:
_out_rank_order_event_raster_debugger.active_epoch_IDX

In [ ]:
_out_rank_order_event_raster_debugger.active_epoch_result_df

In [ ]:
aclu_y_values_dict = {_active_plot_identifier:{int(aclu):new_sorted_raster.neuron_y_pos[aclu] for aclu in new_sorted_raster.neuron_IDs} for _active_plot_identifier, new_sorted_raster in _out_rank_order_event_raster_debugger.plots_data.seperate_new_sorted_rasters_dict.items()}
aclu_max_y_values_dict = {_active_plot_identifier:np.max(list({int(aclu):new_sorted_raster.neuron_y_pos[aclu] for aclu in new_sorted_raster.neuron_IDs}.values())) for _active_plot_identifier, new_sorted_raster in _out_rank_order_event_raster_debugger.plots_data.seperate_new_sorted_rasters_dict.items()} # {'long_LR': 51.48039215686274, 'long_RL': 53.5, 'short_LR': 51.48039215686274, 'short_RL': 53.5}
global_max_y_value = np.max(list(aclu_max_y_values_dict.values()))
global_max_y_value

In [ ]:
max_n_neurons = np.max([len(v) for v in _out_rank_order_event_raster_debugger.plots_data.unsorted_original_neuron_IDs_lists])
max_n_neurons

In [ ]:
_out_rank_order_event_raster_debugger.plots.all_separate_plots['long_LR']['root_plot']


root_plots_dict

In [ ]:
max_n_neurons

In [ ]:


all_separate_root_plots = {a_decoder_name:a_raster_setup_tuple.plots.root_plot for a_decoder_name, a_raster_setup_tuple in rasters_display_outputs.items()}


In [ ]:
_out_rank_order_selected_spikes_only_event_raster_debugger.ui.tableView

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles

_out_LAPS_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, deepcopy(rank_order_results.LR_laps.epochs_df), track_templates, rank_order_results, None, None)
_out_LAPS_rank_order_event_raster_debugger.params.is_laps = True
_out_LAPS_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_update_plot_titles_callback'] = debug_update_plot_titles

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

In [ ]:
# Scroll to the specified row
tableView.scrollTo(tableView.model().index(row_index, 0))

# Select the entire row
tableView.selectRow(row_index)

In [ ]:
_out_rank_order_event_raster_debugger.plots_data.LR_selected_spike_df #.LR_active_epochs_selected_spikes_fragile_linear_neuron_IDX_dict
# _out_rank_order_event_raster_debugger.plots_data.LR_neuron_id_to_new_IDX_map

In [ ]:
_obj = _out_rank_order_event_raster_debugger
## Add the spikes
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.LR_selected_spike_df), _active_plot_identifier = 'long_LR')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.LR_selected_spike_df), _active_plot_identifier = 'short_LR')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.RL_selected_spike_df), _active_plot_identifier = 'long_RL')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.RL_selected_spike_df), _active_plot_identifier = 'short_RL')

In [ ]:
list(_out_rank_order_event_raster_debugger.plots.keys())
_out_rank_order_event_raster_debugger.plots['scatter_plots']['long_LR'].addPoints()



In [ ]:
list(_out_rank_order_event_raster_debugger.plots_data.keys())


In [ ]:
_out_rank_order_event_raster_debugger.plots_data['all_selected_spots_dict']


In [ ]:
_out_rank_order_event_raster_debugger.plots_data['seperate_all_spots_dict']

In [ ]:
_active_plot_identifier = 'long_LR'
new_sorted_raster = _out_rank_order_event_raster_debugger.plots_data['seperate_new_sorted_rasters_dict'][_active_plot_identifier]
# selected_spikes_df = new_sorted_raster.update_spikes_df_visualization_columns(spikes_df=selected_spikes_df)
# selected_spikes_df
new_sorted_raster.spikes_df

In [ ]:
## Add the source data (spikes_df) to the plot_data
plots_data.spikes_df = deepcopy(spikes_df)    
# Update the dataframe
plots_data.spikes_df = new_sorted_raster.update_spikes_df_visualization_columns(spikes_df=plots_data.spikes_df)
## Build the spots for the raster plot:
_out_rank_order_event_raster_debugger.plots_data['all_selected_spots_dict'][_active_plot_identifier], _out_rank_order_event_raster_debugger.plots_data.all_selected_scatterplot_tooltips_kwargs_dict[_active_plot_identifier] = new_sorted_raster.build_spikes_all_spots_from_df(spikes_df=plots_data.spikes_df, should_return_data_tooltips_kwargs=True, generate_debug_tuples=False)


In [ ]:
_out_rank_order_event_raster_debugger.plots_data.spikes_df

In [ ]:
_out_rank_order_event_raster_debugger.global_spikes_df

In [ ]:
add_selected_spikes_df_points_to_scatter_plot

## Connects TemplatesDebugger to RasterDebugger

In [24]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles


# active_plotter = _out_rank_order_event_raster_debugger
active_plotter = _out_rank_order_selected_spikes_only_event_raster_debugger

# _out_directional_template_pfs_debugger = None

def plot_attached_directional_templates_pf_debugger(curr_active_pipeline, rank_order_raster_debugger: RankOrderRastersDebugger):
    """ builds a _display_directional_template_debugger, attaches it to the provided rank_order_event_raster_debugger so it's updated on its callback, and then returns what it created. 
    
    """
    curr_active_pipeline.reload_default_display_functions()
    # epoch_active_aclus = np.array([9,  26,  31,  39,  40,  43,  47,  52,  53,  54,  60,  61,  65,  68,  72,  75,  77,  78,  81,  82,  84,  85,  90,  92,  93,  98, 102]) # some test indicies
    epoch_active_aclus = None
    _out_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=epoch_active_aclus, figure_name=f'<Controlled by RankOrderRastersDebugger>')

    def debug_update_paired_directional_template_pfs_debugger(a_plotter, an_idx: int):
        """ captures: _out_directional_template_pfs_debugger, """
        epoch_active_aclus = deepcopy(a_plotter.get_epoch_active_aclus())
        # update the displayed cells:
        directional_template_pfs_debugger_on_update_callback = _out_directional_template_pfs_debugger.get('ui').on_update_callback
        directional_template_pfs_debugger_on_update_callback(epoch_active_aclus)
    
    rank_order_raster_debugger.on_idx_changed_callback_function_dict['debug_update_paired_directional_template_pfs_debugger'] = debug_update_paired_directional_template_pfs_debugger
    
    return _out_directional_template_pfs_debugger, debug_update_paired_directional_template_pfs_debugger


_out_directional_template_pfs_debugger, debug_update_paired_directional_template_pfs_debugger = plot_attached_directional_templates_pf_debugger(curr_active_pipeline, rank_order_raster_debugger=active_plotter)

active_plotter.on_idx_changed_callback_function_dict['a_debug_callback'] = a_debug_callback_fn
active_plotter.on_idx_changed_callback_function_dict['debug_update_plot_titles_callback'] = debug_update_plot_titles
# active_plotter.on_idx_changed_callback_function_dict['debug_update_paired_directional_template_pfs_debugger'] = debug_update_paired_directional_template_pfs_debugger
# active_plotter.on_idx_changed_callback_function_dict['debug_update_long_short_info_titles'] = debug_update_long_short_info_titles
# active_plotter.on_idx_changed_callback_function_dict['debug_plot_epoch_label_row'] = debug_plot_epoch_label_row

# active_plotter.on_update_epoch_IDX(11)

PhoDockAreaContainingWindow.GlobalConnectionManagerAccessingMixin_on_setup()
PhoDockAreaContainingWindow.try_register_any_control_widgets()
	flat_widgets_list contains 0 items
using overriden dock location.


_tip_fn(...): data_string: t: 582.790511211264|aclu: 9|neuron_IDX: 1|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 618.1566166237462|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 614.3518544117687|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 613.2510652154451|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 618.1566166237462|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 614.3518544117687|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 613.2510652154451|aclu: 93|neuron_IDX: 32|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 636.1149971890263|aclu: 57|neuron_IDX: 15|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 636.1037843947997|aclu: 28|neuron_IDX: 9|visualization_raster_y_location: nan
_tip_fn(...): data_stri

# 🔶 2023-12-20 - Spearman Investigations

In [ ]:
active_plotter.active_epoch_df

In [26]:
active_plotter.active_epoch_result_df

,LR_Long_spearman,RL_Long_spearman,LR_Short_spearman,RL_Short_spearman,LR_Long_pearson,RL_Long_pearson,LR_Short_pearson,RL_Short_pearson,LR_Long_spearman_Z,RL_Long_spearman_Z,LR_Short_spearman_Z,RL_Short_spearman_Z,LR_Long_pearson_Z,RL_Long_pearson_Z,LR_Short_pearson_Z,RL_Short_pearson_Z,label
47,0.642371,0.143608,0.666674,-0.021016,0.669165,0.178174,0.715529,-0.098999,1.548077,0.36326,1.606779,-0.055332,1.610068,0.450702,1.720307,-0.253245,47.0


In [27]:
active_plotter.global_spikes_df

,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,traj,lap,maze_relative_lap,maze_id,is_theta,is_ripple,theta_phase_radians,neuron_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,fragile_linear_neuron_IDX,PBE_id,scISI,neuron_IDX,Probe_Epoch_id
0,1388610.0,643083.191871,42.658077,9,20,80,2,220.085161,150.838133,1.146709,7,1,1,1,True,False,5.030498,NeuronType.PYRAMIDAL,23409,0.760979,0.528578,218.496767,33,0,3.092212,33,0
1,1388850.0,643083.199244,42.665450,1,13,8,2,220.081028,150.845626,1.145837,7,1,1,1,True,False,5.338121,NeuronType.PYRAMIDAL,23418,0.760947,0.528708,218.496291,4,0,19.226778,4,0
2,1389115.0,643083.207385,42.673591,8,6,58,1,220.076569,150.853714,1.145948,7,1,1,1,True,False,5.703985,NeuronType.PYRAMIDAL,23430,0.760956,0.528849,218.495795,24,0,2.039961,24,0
3,1389204.0,643083.210119,42.676325,9,7,72,2,220.075071,150.856430,1.145986,7,1,1,1,True,False,5.801769,NeuronType.PYRAMIDAL,23434,0.760960,0.528897,218.495628,31,0,3.868967,31,0
4,1389311.0,643083.213406,42.679612,8,10,61,2,220.073270,150.859695,1.146031,7,1,1,1,True,False,5.930796,NeuronType.PYRAMIDAL,23438,0.760964,0.528954,218.495427,26,0,3.862301,26,0
5,1389664.0,643083.224250,42.690456,11,4,93,2,220.067330,150.870468,1.146180,7,1,1,1,True,False,0.086853,NeuronType.PYRAMIDAL,23450,0.760979,0.529142,218.494765,42,0,0.105707,42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,56436796.0,644774.271279,1733.737485,8,5,57,2,205.086329,142.299499,0.294023,11,-1,-1,2,False,False,5.139826,NeuronType.PYRAMIDAL,867350,0.715153,0.499978,203.031864,23,420,0.024484,23,409
3132,56437479.0,644774.292261,1733.758467,11,4,93,2,205.088549,142.305429,0.292292,11,-1,-1,2,False,True,0.000115,NeuronType.PYRAMIDAL,867369,0.715122,0.499987,203.035023,42,420,1.081681,42,409
3133,56437503.0,644774.292998,1733.759204,9,20,80,2,205.088627,142.305637,0.292231,11,-1,-1,2,False,True,0.044405,NeuronType.PYRAMIDAL,867370,0.715121,0.499987,203.035134,33,420,2.477260,33,409


In [29]:
curr_epoch_spikes_df = deepcopy(active_plotter.get_active_epoch_spikes_df())
curr_epoch_spikes_df

,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,traj,lap,maze_relative_lap,maze_id,is_theta,is_ripple,theta_phase_radians,neuron_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,fragile_linear_neuron_IDX,PBE_id,scISI,neuron_IDX,Probe_Epoch_id
264,6238055.0,643232.166745,191.632951,8,10,61,2,242.092099,141.631492,2.520449,13,-1,-1,1,False,False,4.828740,NeuronType.PYRAMIDAL,96455,0.842691,0.496986,240.886587,26,48,0.682260,26,47
265,6238435.0,643232.178419,191.644625,8,7,59,2,242.063358,141.626197,2.528076,13,-1,-1,1,False,False,5.528952,NeuronType.PYRAMIDAL,96467,0.842584,0.496965,240.859969,25,48,1.377392,25,47
266,6240692.0,643232.247754,191.713960,12,15,104,2,241.894075,141.595007,2.551611,13,-1,-1,1,False,True,4.519270,NeuronType.PYRAMIDAL,96535,0.841791,0.496836,240.704762,44,48,4.276867,44,47
267,6240752.0,643232.249597,191.715803,10,12,86,2,241.889159,141.594101,2.549803,13,-1,-1,1,False,True,4.623126,NeuronType.PYRAMIDAL,96537,0.841771,0.496841,240.700260,37,48,2.966137,37,47
268,6241200.0,643232.263360,191.729566,9,7,72,2,241.852452,141.587338,2.536309,13,-1,-1,1,False,True,5.456748,NeuronType.PYRAMIDAL,96552,0.841625,0.496875,240.666638,31,48,1.478399,31,47
269,6241488.0,643232.272207,191.738413,1,13,8,2,241.828855,141.582990,2.527634,13,-1,-1,1,False,True,5.964093,NeuronType.PYRAMIDAL,96560,0.841530,0.496900,240.645025,4,48,10.035451,4,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,6243800.0,643232.343232,191.809438,3,37,39,2,241.652413,141.550481,2.548276,13,-1,-1,1,False,True,3.451643,NeuronType.PYRAMIDAL,96628,0.840569,0.497749,240.612804,19,48,0.321024,19,47
279,6243997.0,643232.349284,191.815490,2,21,24,2,241.637235,141.547684,2.551343,13,-1,-1,1,False,True,3.721463,NeuronType.PYRAMIDAL,96634,0.840453,0.497814,240.607325,11,48,4.374219,11,47
280,6244118.0,643232.353001,191.819207,9,19,79,2,241.627855,141.545956,2.552875,13,-1,-1,1,False,True,3.827078,NeuronType.PYRAMIDAL,96638,0.840377,0.497846,240.603535,32,48,4.334682,32,47


In [ ]:
curr_epoch_spikes_df['t_rel_seconds'].corr( , method='spearman')
                          


In [34]:
short_LR_rank_list = track_templates.decoder_LR_pf_peak_ranks_list[0]
short_LR_rank_list

array([22, 36, 38, 33, 23, 24,  8, 40,  2, 32,  7, 51, 37, 17, 26,  1,  3, 20, 42, 41, 13, 10, 21, 12,  5, 45, 25, 16, 34, 47, 43, 30, 48, 18, 39, 50, 14, 31, 15, 28, 46,  6, 44, 35,  4,  9, 11, 27, 19, 49, 29])

In [38]:
## Get the ranks for the active aclus in the epoch:
short_LR_peak_x_map = deepcopy(active_plotter.track_templates.get_decoder_aclu_peak_maps().short_LR)


In [43]:
curr_epoch_spikes_df.aclu.map(short_LR_peak_x_map) # 2023-12-20 - The core issue seems to involve missing values. For example, when an aclu is missing from this template, a NaN is returned.

## ERROR HERE: 
# 280: aclu=79, result=122.208089
# 281: aclu=48, result=122.208089
# WHY WOULD THEY BE THE SAME?
short_LR_peak_x_map[79]

122.20808903087519

In [45]:
short_LR_peak_x_map[48]

122.20808903087519

In [ ]:
# corrwith
scipy.stats.spearmanr(active_epoch_aclu_short_ranks, actually_included_epoch_ranks) # Both arrays need to have the same length in the axis dimension.

In [ ]:
active_epochs_df = deepcopy(ripple_result_tuple.active_epochs)
active_epochs_df

In [ ]:
active_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
active_epochs_df

In [ ]:
print(list(rank_order_raster_debugger.global_spikes_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']



In [ ]:
print(list(active_selected_spikes_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']



In [ ]:
print(list(active_epochs_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']


a_row, a_label = get_epoch_label_row(188)
a_row



In [ ]:
a_row = list(a_row.itertuples())[0]

In [ ]:
list(rank_order_raster_debugger.plots_data.keys())


In [ ]:
debug_update_plot_titles(rank_order_raster_debugger, an_idx=20)

In [ ]:
a_debug_callback_fn(rank_order_raster_debugger, an_idx=20)

In [ ]:
list(rank_order_raster_debugger.plots.root_plots.keys())


In [ ]:
directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = ripple_result_tuple.directional_likelihoods_tuple
directional_likelihoods_tuple.long_best_direction_indices
ripple_result_tuple.rank_order_z_score_df

In [ ]:
ripple_result_tuple.rank_order_z_score_df.label

### Independent DirectionalTemplatePFsDebugger for testing

In [53]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.TemplateDebugger import TemplateDebugger

# _display_directional_laps_overview:
curr_active_pipeline.prepare_for_display()
curr_active_pipeline.reload_default_display_functions()
session_description: str = curr_active_pipeline.get_session_context().get_description()
print(f'session_description: {session_description}')
pg.setConfigOptions(imageAxisOrder='row-major')
# _out = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_laps_overview)
_out_all_cells_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=None, figure_name='All Cells (Independent)')
all_cells_directional_template_pfs_debugger: TemplateDebugger = _out_all_cells_directional_template_pfs_debugger['obj']

[autoreload of pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.TemplateDebugger failed: Traceback (most recent call last):
  File "k:\FastSwap\Environments\pypoetry\pypoetry\Cache\virtualenvs\spike3d-UP7QTzFM-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "k:\FastSwap\Environments\pypoetry\pypoetry\Cache\virtualenvs\spike3d-UP7QTzFM-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "k:\FastSwap\Environments\pypoetry\pypoetry\Cache\virtualenvs\spike3d-UP7QTzFM-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "k:\FastSwap\Environments\pypoetry\pypoetry\Cache\virtualenvs\spike3d-UP7QTzFM-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "k:\FastSwap\Environments\pypoetry\pypo

session_description: kdiba_gor01_one_2006-6-09_1-22-43
WARN: 2023-12-11 - enable_pf_peak_indicator_lines is not yet implemented and the lines are not correctly aligned.
PhoDockAreaContainingWindow.GlobalConnectionManagerAccessingMixin_on_setup()
PhoDockAreaContainingWindow.try_register_any_control_widgets()
	flat_widgets_list contains 0 items
using overriden dock location.


In [ ]:
plots = _out_all_cells_directional_template_pfs_debugger['plots']
a_plot_widget, an_image_item = plots['pf1D_heatmaps']['long_LR'] # (pyqtgraph.widgets.PlotWidget.PlotWidget, pyqtgraph.graphicsItems.ImageItem.ImageItem)
a_view_box = an_image_item.getViewBox()
a_plot_widget


In [51]:
all_cells_directional_template_pfs_debugger.plots_data.sorted_pf_peak_location_list


[array([9.31427, 21.817, 18.708, 11.561, 14.1151, 10.0726, 16.6196, 9.01655, 7.45477, 5.75679, 4.85872, 5.76916, 7.94376, 24.9744, 12.1798, 15.8822, 18.0797, 18.7215, 18.4351, 19.5929, 20.4058, 21.6332, 23.8594, 26.0069, 26.3567, 22.3504, 24.9917, 24.6407, 30.5369, 33.1472, 33.5163, 32.3855, 32.783, 36.219, 34.7201, 37.7695, 28.6427, 30.9646, 32.0758, 37.7068, 46.3524, 39.1924, 46.3567, 42.9213, 46.0885, 47.9163, 51.8735, 39.7348, 40.9617, 47.7288, 50.7682]),
 array([6.53038, 11.9473, 3.83609, 11.9134, 20.1915, 17.2963, 6.79891, 8.47176, 7.70657, 13.6991, 11.7591, 21.0338, 18.5646, 17.0268, 23.6599, 28.6749, 22.6886, 24.0178, 26.6566, 29.2688, 30.1285, 26.5089, 28.5347, 30.2055, 33.3237, 32.3055, 32.594, 30.0889, 33.0648, 34.1043, 32.6016, 26.0982, 30.3565, 30.4955, 34.7694, 31.0183, 38.5724, 40.9223, 48.2432, 47.5621, 50.0379, 47.3669, 51.6823, 52.3119, 48.4216, 50.0238, 39.1445, 50.5264, 35.8459, 33.8935, 43.7168, 39.2466, 42.0123, 36.7137, 52.3817, 48.268]),
 array([15.5624, 16.1674

In [ ]:
from PendingNotebookCode import PyQtGraphCrosshairs





In [ ]:
.data.sorted_pf_peak_location_list

In [ ]:
## Just compute line point locations... EZ.


In [ ]:
rois = []
rois.append(pg.RectROI([20, 20], [20, 20], pen=(0,9)))


In [ ]:

# Function to handle hover event
def hoverEvent(pos):
    print(f'hoverEvent')
    index = int(pos.x())
    region.setRegion([index, index + 1])  # Highlight the entire row

# Add LinearRegionItem for highlighting
region = pg.LinearRegionItem(brush='#FFFFFF', hoverBrush='#FFFFFF', movable=False, clipItem=an_image_item)
region.setZValue(10)

a_plot_widget.addItem(region, ignoreBounds=True)

# Connect hover event
a_plot_widget.scene().sigMouseMoved.connect(hoverEvent)

region.setRegion([0.0, 10.0 + 1])

In [ ]:
# Set spike emphasis on debug rasters?

# Get debug rasters to match the colors in template viewer?

from numpy import ma

def _perform_compute_directional_likelihoods_tuple_methods(active_epochs, active_LR_long_z_score=None, active_RL_long_z_score=None, active_LR_short_z_score=None, active_RL_short_z_score=None) -> DirectionalRankOrderLikelihoods:
	""" Used to switch between three different temporary methods of determining the proper direction.
	captures curr_active_pipeline, global_epoch_name """
	# ALT_METHOD : main `_compute_best_direction_likelihoods`
	# #TODO 2023-12-12 11:56: - [ ] Should these be saved to the ripple result?
	# spikes_df = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].spikes_df)
	# # Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
	# (long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
	# return cls._compute_best_direction_likelihoods(active_epochs, long_LR_results, long_RL_results, short_LR_results, short_RL_results, spikes_df, decoding_time_bin_size)

	## ALT 2023-12-12 - APPROXIMATE best-direction without using the decoder as a workaround since `cls._compute_best_direction_likelihoods` isn't working yet:
	print(f'WARNING: approximating best-direction without using the decoder as a workaround.')

	# ALT_METHOD: mean_determined_epoch_dir: `mean_determined_epoch_dir`
	# mean_LR_z_scores = ((np.abs(rank_order_results.LR_ripple.long_z_score) + np.abs(rank_order_results.LR_ripple.short_z_score)) / 2.0)
	# mean_RL_z_scores = ((np.abs(rank_order_results.RL_ripple.long_z_score) + np.abs(rank_order_results.RL_ripple.short_z_score)) / 2.0)
	# mean_determined_epoch_dir = (mean_LR_z_scores >= mean_RL_z_scores).astype(int)
	# long_best_direction_indicies = mean_determined_epoch_dir.copy()
	# short_best_direction_indicies = mean_determined_epoch_dir.copy()

	# ALT_METHOD: best max method:
	# filtered_active_epochs, (active_LR_long_z_score, active_RL_long_z_score, active_LR_short_z_score, active_RL_short_z_score) = rank_order_results.get_aligned_events(active_epochs)
	long_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_long_z_score), np.abs(active_RL_long_z_score)]), axis=0).astype(int)
	short_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_short_z_score), np.abs(active_RL_short_z_score)]), axis=0).astype(int)

	## build the new directional_likelihods_tuple:
	#TODO 2023-12-12 13:50: - [ ] Add `filtered_active_epochs` to the DirectionalRankOrderLikelihoods?
	return DirectionalRankOrderLikelihoods(long_relative_direction_likelihoods=None, short_relative_direction_likelihoods=None, long_best_direction_indices=long_best_direction_indicies, short_best_direction_indices=short_best_direction_indicies)


## Post-process Z-scores with their most likely directions:
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']

ripple_combined_epoch_stats_df = deepcopy(rank_order_results.ripple_combined_epoch_stats_df)
active_replay_epochs = deepcopy(rank_order_results.LR_ripple.epochs_df)

_traditional_arg_dict_key_names = ('active_LR_long_z_score', 'active_RL_long_z_score', 'active_LR_short_z_score', 'active_RL_short_z_score')
_traditional_arg_dict_values_tuple = (ripple_combined_epoch_stats_df.LR_Long_spearman_Z, ripple_combined_epoch_stats_df.RL_Long_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z)
active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score = ripple_combined_epoch_stats_df.LR_Long_spearman_Z, ripple_combined_epoch_stats_df.RL_Long_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z


_arg_dict_from_ripple_combined_epoch_stats_df = dict(zip(_traditional_arg_dict_key_names, _traditional_arg_dict_values_tuple))
ripple_directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = _perform_compute_directional_likelihoods_tuple_methods(None, **_arg_dict_from_ripple_combined_epoch_stats_df)
long_relative_direction_likelihoods, short_relative_direction_likelihoods, long_best_direction_indicies, short_best_direction_indicies = ripple_directional_likelihoods_tuple


ripple_evts_long_best_dir_z_score_values = np.where(long_best_direction_indicies, active_LR_ripple_long_z_score, active_RL_ripple_long_z_score)
ripple_evts_short_best_dir_z_score_values = np.where(short_best_direction_indicies, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score)
ripple_evts_long_short_best_dir_z_score_diff_values = ripple_evts_long_best_dir_z_score_values - ripple_evts_short_best_dir_z_score_values

_LR_INDEX = 0
_RL_INDEX = 1
ripple_masked_z_score_values_list: List[ma.masked_array] = [ma.masked_array(x, mask=np.logical_not(a_mask)) for x, a_mask in zip((active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score),
																									((long_best_direction_indicies == _LR_INDEX), (long_best_direction_indicies == _RL_INDEX), (short_best_direction_indicies == _LR_INDEX), (short_best_direction_indicies == _RL_INDEX)))]


# outputs: ripple_evts_long_short_best_dir_z_score_diff_values
ripple_result_tuple: DirectionalRankOrderResult = DirectionalRankOrderResult(active_replay_epochs, long_best_dir_z_score_values=ripple_evts_long_best_dir_z_score_values, short_best_dir_z_score_values=ripple_evts_short_best_dir_z_score_values,
																			long_short_best_dir_z_score_diff_values=ripple_evts_long_short_best_dir_z_score_diff_values, directional_likelihoods_tuple=ripple_directional_likelihoods_tuple,
																			masked_z_score_values_list=ripple_masked_z_score_values_list, rank_order_z_score_df=None)



In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

## Main
ripple_result_tuple, laps_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.003)
ripple_result_tuple

In [ ]:
laps_result_tuple

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple, curr_active_pipeline.global_computation_results.computed_data['RankOrder'].laps_most_likely_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.006) # 6ms bins

# Plot the z-scores differences and their raw-values

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import plot_rank_order_epoch_inst_fr_result_tuples

ripple_outputs = plot_rank_order_epoch_inst_fr_result_tuples(curr_active_pipeline, ripple_result_tuple, 'Ripple')

In [ ]:
# Usage of the function for Lap
lap_outputs = plot_rank_order_epoch_inst_fr_result_tuples(curr_active_pipeline, laps_result_tuple, 'Lap')

In [ ]:
# ripple_result_tuple.directional_likelihoods_tuple.long_best_direction_indices

# Change column type to int8 for columns: 'long_best_direction_indices', 'short_best_direction_indices'
directional_likelihoods_df = pd.DataFrame.from_dict(ripple_result_tuple.directional_likelihoods_tuple._asdict()).astype({'long_best_direction_indices': 'int8', 'short_best_direction_indices': 'int8'})
directional_likelihoods_df

In [ ]:
directional_likelihoods_df.plot.bar(y=['long_relative_direction_likelihoods', 'short_relative_direction_likelihoods'])

In [ ]:
df = deepcopy(directional_likelihoods_df)

fig = plt.figure(num='directional_likelihoods_df Matplotlib figure')
plt.plot(df.index, df["long_relative_direction_likelihoods"], label="Long Direction")
plt.plot(df.index, df["short_relative_direction_likelihoods"], label="Short Direction")

for i, idx in enumerate(df["long_best_direction_indices"]):
  if idx == 0:
    plt.annotate("↑", (df.index[i], df["long_relative_direction_likelihoods"][i]), textcoords="offset points", xytext=(0, 10))
  elif idx == 1:
    plt.annotate("↓", (df.index[i], df["long_relative_direction_likelihoods"][i]), textcoords="offset points", xytext=(0, -10))

plt.xlabel("Index")
plt.ylabel("Likelihood")
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

# directional_likelihoods_df = pd.DataFrame({
#   "long_relative_direction_likelihoods": [0.41, 0.48, 0.27, 0.33, 0.69, 0.50],
#   "short_relative_direction_likelihoods": [0.58, 0.51, 0.72, 0.66, 0.30, 0.49],
#   "long_best_direction_indices": [0, 1, 1, 1, 0, 0]
# })


fig = plt.figure(num='directional_likelihoods_df figure')
sns.scatterplot(x=directional_likelihoods_df.index, y=directional_likelihoods_df["long_relative_direction_likelihoods"], hue=directional_likelihoods_df["long_best_direction_indices"], palette="hls")
plt.show()

In [ ]:
an_idx: int = 154
# a_label = lookup_label_from_index(an_idx)
# a_label

In [ ]:
active_selected_spikes_df[active_selected_spikes_df['Probe_Epoch_id'] == an_idx]['aclu'].count()

In [ ]:
active_selected_spikes_df[active_selected_spikes_df['Probe_Epoch_id'] == an_idx].plot.scatter(x='t_rel_seconds', y='LR_Long_pf_peak_x')

In [ ]:
## filter events by (|z| > 1.0)
ripple_result_tuple.active_epochs
ripple_result_tuple.long_best_dir_z_score_values

In [ ]:
# ripple_result_tuple.directional_likelihoods_tuple
ripple_result_tuple.directional_likelihoods_tuple.long_best_direction_indices[105] # 1, 

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.short_best_direction_indices[105]

In [ ]:
ripple_result_tuple.long_best_dir_z_score_values[105]

In [ ]:
ripple_result_tuple.short_best_dir_z_score_values[105]

In [ ]:
ripple_result_tuple.active_epochs

In [ ]:
np.shape(ripple_result_tuple.long_best_dir_z_score_values)

In [ ]:
np.shape(ripple_result_tuple.short_best_dir_z_score_values)

In [ ]:
global_replays.n_epochs

In [ ]:
global_replays

In [ ]:
ratemap = long_pf1D.ratemap
ratemap.tuning_curve_unsmoothed_peak_firing_rates

In [ ]:
type(rank_order_results)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderResult

# type(ripple_result_tuple)
ripple_result_tuple.plot_histograms()

In [ ]:
laps_result_tuple.plot_histograms()

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.long_relative_direction_likelihoods.shape

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.short_relative_direction_likelihoods.shape

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.long_best_direction_indices

In [ ]:
ripple_result_tuple.short_best_dir_z_score_values

In [ ]:
# 2023-11-16_LapsRankOrderHistogram Figure:
pd.DataFrame({'long_z_scores': laps_result_tuple.long_best_dir_z_score_values, 'short_z_scores': laps_result_tuple.short_best_dir_z_score_values}).hist()

# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:


In [ ]:
curr_active_pipeline.prepare_for_display()

In [ ]:
# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
# active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.display('_display_spike_rasters_pyqtplot_2D', 'maze_any') # 'maze_any'


In [ ]:
active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()


In [ ]:
spike_raster_window

In [ ]:
spikes_window = spike_raster_window.spikes_window # SpikesDataframeWindow

bottomPlaybackControlBarWidget = spike_raster_window.ui.bottomPlaybackControlBarWidget # Spike3DRasterBottomPlaybackControlBar 

doubleSpinBox_ActiveWindowStartTime = bottomPlaybackControlBarWidget.ui.doubleSpinBox_ActiveWindowStartTime
doubleSpinBox_ActiveWindowEndTime = bottomPlaybackControlBarWidget.ui.doubleSpinBox_ActiveWindowEndTime


# spikes_window.timeWindow.start
# spikes_window.active_window_start_time
# spikes_window.update_window_start_end(451.8908457518555, 451.9895490613999) ## Works but does not trigger refresh/update of the window. The changes are reflected as soon as you try to scroll at all though.
# spikes_window.active_window_end_time

print(f'spikes_window.active_window_start_time: {spikes_window.active_window_start_time}, spikes_window.active_window_end_time: {spikes_window.active_window_end_time}')
# need to block signals:
# doubleSpinBox_ActiveWindowStartTime.blockSignals(True)
# doubleSpinBox_ActiveWindowEndTime.blockSignals(True)
doubleSpinBox_ActiveWindowStartTime.setValue(spikes_window.active_window_start_time)
doubleSpinBox_ActiveWindowEndTime.setValue(spikes_window.active_window_end_time)
# doubleSpinBox_ActiveWindowStartTime.blockSignals(False) # unblock the signals when done
# doubleSpinBox_ActiveWindowEndTime.blockSignals(False)


# @pyqtExceptionPrintingSlot(float, float)
def on_active_window_changed(start_t, end_t, _obj):
	# need to block signals:
	# doubleSpinBox_ActiveWindowStartTime.blockSignals(True)
	# doubleSpinBox_ActiveWindowEndTime.blockSignals(True)
	if start_t is not None:
		doubleSpinBox_ActiveWindowStartTime.setValue(start_t)
	if end_t is not None:
		doubleSpinBox_ActiveWindowEndTime.setValue(end_t)
	# doubleSpinBox_ActiveWindowStartTime.blockSignals(False) # unblock the signals when done
	# doubleSpinBox_ActiveWindowEndTime.blockSignals(False)

curr_window_ctrls_connection = spikes_window.windowed_data_window_updated_signal.connect(on_active_window_changed)


In [ ]:
doubleSpinBox_ActiveWindowStartTime.setReadOnly(True)
doubleSpinBox_ActiveWindowEndTime.setReadOnly(True)

spikes_window.on_window_changed.connect(

In [ ]:
doubleSpinBox_ActiveWindowStartTime.setVisible(False)
bottomPlaybackControlBarWidget.setVisible(False)

In [ ]:
# global_epoch_context
curr_active_pipeline.reload_default_display_functions()
# curr_active_pipeline.prepare_for_display()
curr_active_pipeline.clear_display_outputs()


In [ ]:
# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)

if len(found_spike_raster_windows) < 1:
	# no existing spike_raster_windows. Make a new one
	print(f'no existing SpikeRasterWindow. Creating a new one.')
	# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
	# active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

	active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

else:
	print(f'found {len(found_spike_raster_windows)} existing Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...). Will use the most recent.')
	# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
	# Get the most recent existing one and reuse that:
	spike_raster_window = found_spike_raster_windows[0]


# Extras:
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
main_graphics_layout_widget = active_2d_plot.ui.main_graphics_layout_widget # GraphicsLayoutWidget
main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
background_static_scroll_plot_widget = active_2d_plot.plots.background_static_scroll_window_plot # PlotItem

In [ ]:
spike_raster_window.isVisible() # False
# spike_raster_window.show()
spike_raster_window.close()

In [ ]:
spike_raster_window.connection_man.active_connections

In [ ]:
found_any_window = TopLevelWindowHelper.top_level_windows(pg.mkQApp())
found_any_window

In [ ]:
# print windows:
[print_widget_hierarchy(v) for k, v in found_any_window.items()]

In [ ]:
_display_out = curr_active_pipeline.last_added_display_output


In [ ]:
ipspikesDataExplorer = _display_out['ipspikesDataExplorer']
pActiveSpikesBehaviorPlotter = _display_out['plotter']


In [ ]:
 = curr_active_pipeline.last_added_display_output
_display_out


In [ ]:
ipspikesDataExplorer = self._display_output['ipspikesDataExplorer']

In [ ]:
### Adjusting Spike Emphasis:
#### Usage Examples:
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState
from neuropy.core.neuron_identities import NeuronType


In [ ]:

## Example 1: De-emphasize spikes excluded from the placefield calculations:
is_spike_included_in_pf = np.isin(spike_raster_window.spike_raster_plt_2d.spikes_df.index, active_pf_2D.filtered_spikes_df.index)
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included_in_pf), SpikeEmphasisState.Deemphasized)

## Example 2: De-emphasize spikes that don't have their 'aclu' from a given set of indicies:
is_spike_included = spike_raster_window.spike_raster_plt_2d.spikes_df.aclu.to_numpy() == 2
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included), SpikeEmphasisState.Deemphasized)

## Example 3: De-emphasize all spikes 
active_2d_plot.update_spike_emphasis(new_emphasis_state=SpikeEmphasisState.Deemphasized)

## Example 4: Hide all spikes entirely
active_2d_plot.update_spike_emphasis(new_emphasis_state=SpikeEmphasisState.Hidden)


In [ ]:
## Setup: Hide all non-pyramidal spikes entirely
spikes_df = spike_raster_window.spikes_df
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not((spikes_df.neuron_type == NeuronType.from_string('pyr'))), SpikeEmphasisState.Hidden)

In [ ]:
spikes_window = spike_raster_window.spikes_window # SpikesDataframeWindow
# spikes_window.update_window_start_end(451.8908457518555, 451.9895490613999) ## Works but does not trigger refresh/update of the window. The changes are reflected as soon as you try to scroll at all though.


In [ ]:
# 20*60.0 + 50.0 +  0.218 = 1250.218

spikes_window.update_window_start_end(1250.218, (1250.218 + 3.0))



In [ ]:
spikes_window.window_duration # Prints the current window's duration. The win. dur. label control in the left bar is not updated.

desired_window_fraction: float = 0.1 # 10% of the window is the default jump size
relevant_jump_duration: float = spikes_window.window_duration * desired_window_fraction
relevant_jump_duration


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.spikes_mixins import SpikeRenderingPyVistaMixin
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

found_windows_of_type = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=InteractivePlaceCellDataExplorer)
found_windows_of_type
TopLevelWindowHelper.top_level_windows(pg.mkQApp(), only_visible=True)

In [ ]:
(451.8908457518555, 451.9895490613999)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import BatchPhoJonathanFiguresHelper

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True, disable_top_row=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}

# PhoKamran2023Paper Results

In [ ]:
pg.setConfigOptions(background='white', foreground='black') # black on white background (more traditional) color scheme

## Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters

curr_active_pipeline.prepare_for_display()
pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics, fig_1c_figures_out_dict = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
# rdf = jonathan_firing_rate_analysis_result.rdf.rdf
# rdf
# ==================================================================================================================== #
# Fig 1c) 2023-07-14 - LxC and SxC PhoJonathanSession plots                                                            #
# ==================================================================================================================== #
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis.neuron_replay_stats_df, curr_jonathan_firing_rate_analysis.rdf.rdf, curr_jonathan_firing_rate_analysis.rdf.aclu_to_idx, curr_jonathan_firing_rate_analysis.irdf.irdf

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True, disable_top_row=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}


## Figure 2) `PaperFigureTwo`: LxC/SxC Analyses
Note: this fails when SxC or LxC are empty for this session (as it's not meaningful to produce a comparison bar plot). In this case, aggregate across multiple sessions.

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline)
_out_fig_2.display()

## Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
# curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

##  All Programmatic Plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots


batch_perform_all_plots(curr_active_pipeline, enable_neptune=False, neptuner=None)

# Simple ActiveSet Directional Determination

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderResult
from pyphocorehelpers.indexing_helpers import bidirectional_setdiff1d

# rank_order_results.LR_ripple.extra_info_dict

# active_epochs_df


In [ ]:
bidirectional_setdiff1d(rank_order_results.LR_ripple.epochs_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy())

In [ ]:
# spikes_df.flat_spike_idx
# np.setdiff1d(rank_order_results.LR_ripple.spikes_df.flat_spike_idx.to_numpy(), rank_order_results.RL_ripple.spikes_df.flat_spike_idx.to_numpy())
bidirectional_setdiff1d(rank_order_results.LR_ripple.spikes_df.flat_spike_idx.to_numpy(), rank_order_results.RL_ripple.spikes_df.flat_spike_idx.to_numpy())

In [ ]:
display(bidirectional_setdiff1d(rank_order_results.LR_ripple.spikes_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy()))
display(np.setdiff1d(rank_order_results.LR_ripple.spikes_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy()))

In [ ]:
rank_order_results.RL_ripple.epochs_df

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer
# from PendingNotebookCode import epoch_directionality_active_set_evidence
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderLikelihoods

# active_epochs_df.label.astype(label_column_type).map(lambda x: rank_order_results.LR_ripple.epoch_template_active_aclus[x])

# ['LR_Long_ActuallyIncludedAclus', 'RL_Long_ActuallyIncludedAclus', 'LR_Short_ActuallyIncludedAclus', 'RL_Short_ActuallyIncludedAclus']

# rank_order_results.LR_laps.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.LR_laps.epochs_df, is_laps=True)
# rank_order_results.RL_laps.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.RL_laps.epochs_df, is_laps=True)

# rank_order_results.LR_ripple.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.LR_ripple.epochs_df, is_laps=False)
# rank_order_results.RL_ripple.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.RL_ripple.epochs_df, is_laps=False)

rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_best_direction_indices

In [ ]:
def _validate_estimated_lap_dirs():
	""" 2023-12-19 - validstes the estimated lap directions against the ground-truth direction which is known for the laps. 
 	"""
	lap_dir_is_LR = deepcopy(rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_best_direction_indices)
	lap_dir_index = deepcopy(lap_dir_is_LR).astype('int8')

	# global_laps
	actual_lap_dir = deepcopy(global_any_laps_epochs_obj.to_dataframe()['lap_dir'])
	n_total_laps = np.shape(actual_lap_dir)[0]
	is_correct = (actual_lap_dir.values == lap_dir_index)
	n_correct_direction = is_correct.sum()
	n_correct_direction

	print(f'Lap directions: {n_correct_direction}/{n_total_laps} correct ({100.0*float(n_correct_direction)/float(n_total_laps)}%)') # Lap directions: 76/80 correct (95.0%)


In [ ]:
rank_order_results.adding_active_aclus_info()
rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.006) # 6ms bins
        
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_relative_direction_likelihoods

In [ ]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.short_best_direction_indices


In [ ]:


long_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_long_z_score), np.abs(active_RL_long_z_score)]), axis=0).astype(int)
short_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_short_z_score), np.abs(active_RL_short_z_score)]), axis=0).astype(int)

In [ ]:
recovered_lap_epochs = rank_order_results.laps_most_likely_result_tuple.active_epochs
estimated_directions = deepcopy(rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_best_direction_indices)
assert len(recovered_lap_epochs) == len(estimated_directions), f"recovered_lap_epochs should be same length as estimated_directions"
estimated_directions

In [ ]:
recovered_lap_epochs['estimated_direction_index'] = estimated_directions
recovered_lap_epochs

In [ ]:
recovered_lap_epochs

In [ ]:
rank_order_results.LR_laps.epochs_df

In [ ]:
rank_order_results.laps_most_likely_result_tuple.plot_histograms()


In [ ]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple

In [ ]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple
, 


In [ ]:
RL_laps_epochs_df

In [ ]:
rank_order_results.LR_ripple.epochs_df

In [ ]:
epochs_df_L.plot.scatter(x='start', y='normed_LR_evidence') # 'normed_product_LR_evidence'
epochs_df_L.plot.scatter(x='start', y='normed_product_LR_evidence') # 'normed_product_LR_evidence'

In [ ]:
rank_order_results.LR_laps.epochs_df

In [ ]:
rank_order_results.RL_laps.epochs_df

In [ ]:
ActuallyIncludedAclusColumnNames = ['LR_Long_ActuallyIncludedAclus', 'RL_Long_ActuallyIncludedAclus', 'LR_Short_ActuallyIncludedAclus', 'RL_Short_ActuallyIncludedAclus']
LR_laps_epochs_df[ActuallyIncludedAclusColumnNames]

## find uniques in each one:

def uniques(*aclus_lists):
    

In [ ]:
an_aclus_lists = LR_laps_epochs_df.loc[0, ActuallyIncludedAclusColumnNames].to_dict()
# {'LR_Long_ActuallyIncludedAclus': array([  5,  17,  25,  31,  32,  34,  36,  41,  45,  48,  50,  53,  54,  55,  57,  58,  59,  61,  62,  63,  64,  66,  68,  69,  74,  75,  76,  78,  81,  82,  83,  84,  86,  87,  88,  89,  90,  92,  93,  96,  98, 100, 102, 108]),
#  'RL_Long_ActuallyIncludedAclus': array([  3,   5,  10,  11,  14,  15,  16,  21,  31,  32,  33,  35,  36,  37,  41,  45,  48,  49,  50,  53,  55,  57,  59,  60,  61,  62,  63,  64,  69,  70,  71,  73,  75,  76,  78,  81,  83,  84,  86,  88,  89,  90,  92,  93,  98, 100, 102, 107, 108]),
#  'LR_Short_ActuallyIncludedAclus': array([  5,  17,  25,  31,  32,  34,  36,  41,  45,  48,  50,  53,  54,  55,  57,  58,  59,  61,  62,  63,  64,  66,  68,  69,  74,  75,  76,  78,  81,  82,  83,  84,  86,  87,  88,  89,  90,  92,  93,  96,  98, 100, 102, 108]),
#  'RL_Short_ActuallyIncludedAclus': array([  3,   5,  10,  11,  14,  15,  16,  21,  31,  32,  33,  35,  36,  37,  41,  45,  48,  49,  50,  53,  55,  57,  59,  60,  61,  62,  63,  64,  69,  70,  71,  73,  75,  76,  78,  81,  83,  84,  86,  88,  89,  90,  92,  93,  98, 100, 102, 107, 108])}
# an_aclus_lists['LR_Long_ActuallyIncludedAclus']




LR_only, RL_only = bidirectional_setdiff1d(an_aclus_lists['LR_Long_ActuallyIncludedAclus'], an_aclus_lists['RL_Long_ActuallyIncludedAclus'])




In [ ]:
[bidirectional_setdiff1d(a_LR_aclus, a_RL_aclus) for a_LR_aclus, a_RL_aclus in zip(LR_laps_epochs_df['LR_Long_ActuallyIncludedAclus'].to_list(), LR_laps_epochs_df['RL_Long_ActuallyIncludedAclus'].to_list())]


In [ ]:
RL_laps_epochs_df

In [ ]:
# epochs_df_L.plot(x='start', y=['LR_evidence', 'LR_product_evidence'])

In [ ]:
epochs_df_L.plot(x='start', y=['normed_LR_evidence', 'normed_product_LR_evidence'])

In [ ]:
rank_order_results.LR_laps.epochs_df

In [ ]:
epoch_accumulated_evidence, epoch_rate_dfs, epochs_df_L = epoch_directionality_active_set_evidence(decoders_dict, active_epochs_df)
epochs_df_L


In [ ]:
# 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Short_ActuallyIncludedAclus', 'LR_Short_rel_num_cells', 'RL_Short_ActuallyIncludedAclus', 'RL_Short_rel_num_cells'
# 
# epoch_rate_dfs

['normed_LR_evidence','normed_RL_evidence','normed_product_LR_evidence','normed_product_RL_evidence']

In [ ]:
long_laps.to_dataframe()

In [ ]:
epoch_rate_df = epoch_rate_dfs[410]
epoch_rate_df

In [ ]:
epoch_accumulated_evidence

In [ ]:
epochs_df_L['normed_LR_evidence'].hist()

# epochs_df_L[['LR_evidence', 'RL_evidence']]/epochs_df_L[['LR_evidence', 'RL_evidence']].sum(axis=1)


In [ ]:
epochs_df_L['normed_product_LR_evidence'].hist()
epochs_df_L['normed_product_RL_evidence'].hist()


In [ ]:
epoch_rate_dfs[188]

In [ ]:
epochs_df_L.plot(x='start', y='normed_LR_evidence')

In [ ]:
# Determine which cells are active in the given epoch:

# Get these cells' firing rates, using 0.0 if it isn't included in the map for one of the directions




In [ ]:
ripple_combined_epoch_stats_df

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.index).any()

In [ ]:
np.isnan(rank_order_results.ripple_combined_epoch_stats_df.index)

In [ ]:
np.isnan(rank_order_results.ripple_combined_epoch_stats_df.label.to_numpy()).all()

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df

# 2023-12-18 - Simpily detect bimodal cells:

In [ ]:
long_pf1D.peak_indicies

In [ ]:

# active_ratemap.n_neurons


In [ ]:

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
curr_active_pipeline.display('_display_1d_placefields', 'maze1_any')


In [ ]:
from scipy.signal import find_peaks

def compute_tuning_curve_modes(active_ratemap):
    """ 2023-12-19 - Uses `scipy.signal.find_peaks to find the number of peaks or ("modes") for each of the cells in the ratemap. 
    
    Usage:    
		active_ratemap = deepcopy(long_LR_pf1D.ratemap)
		peaks_dict, aclu_n_peaks_dict, unimodal_peaks_dict = compute_tuning_curve_modes(active_ratemap)
		aclu_n_peaks_dict

    """
	active_tuning_curves = deepcopy(active_ratemap.tuning_curves)
	# active_ratemap.tuning_curves.shape # (73, 56) - (n_neurons, n_pos_bins)
	peaks_list = [find_peaks(active_tuning_curves[i,:])[0] for i in np.arange(active_ratemap.n_neurons)] # [0] outside the find_peaks function gets the location of the peak
	peaks_dict = dict(zip(active_ratemap.neuron_ids, peaks_list))
	aclu_n_peaks_dict = {k:len(v) for k,v in peaks_dict.items()} # number of peaks ("models" for each aclu)
	unimodal_peaks_dict = {k:v for k,v in peaks_dict.items() if len(v) < 2}
	return peaks_dict, aclu_n_peaks_dict, unimodal_peaks_dict


# active_ratemap = deepcopy(long_pf1D.ratemap)
active_ratemap = deepcopy(long_LR_pf1D.ratemap)
peaks_dict, aclu_n_peaks_dict, unimodal_peaks_dict = compute_tuning_curve_modes(active_ratemap)
aclu_n_peaks_dict

In [ ]:
aclu_n_peaks_dict

In [ ]:
unimodal_only_aclus = np.array(list(unimodal_peaks_dict.keys()))
unimodal_only_aclus

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze1_any', included_unit_neuron_IDs=unimodal_only_aclus, sortby=[0, 1])

## 2023-12-19

5 epochs where we have a higher Z for short than we do for long

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 148]

In [ ]:
rank_order_results.LR_ripple.epochs_df[rank_order_results.LR_ripple.epochs_df.label == 148]
# rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 148]

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman'].abs() > rank_order_results.ripple_combined_epoch_stats_df['LR_Long_spearman'].abs()]

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[(rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman_Z'].abs() > rank_order_results.ripple_combined_epoch_stats_df['LR_Long_spearman_Z'].abs()) & (rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman_Z'].abs() > 1.98)]

In [ ]:
## Get old LR_ripple/RL_ripple results
LR_active_result: RankOrderResult = rank_order_results.LR_ripple
LR_active_result_epochs = LR_active_result.epochs_df
LR_active_result_epoch_labels = LR_active_result.epochs_df.label.to_numpy()
LR_active_result_n_epochs = np.shape(LR_active_result.epochs_df)[0]
LR_z_variable_shapes = [np.shape(x) for x in (LR_active_result.long_z_score, LR_active_result.short_z_score, LR_active_result.long_short_z_score_diff)]
assert np.all([LR_active_result_n_epochs == s[0] for s in LR_z_variable_shapes]), f"z-variable shapes should match the number of epochs"


RL_active_result: RankOrderResult = rank_order_results.RL_ripple
RL_active_result_epochs = RL_active_result.epochs_df
RL_active_result_epoch_labels = RL_active_result.epochs_df.label.to_numpy()
print(f'active_result_epoch_labels: {RL_active_result_epoch_labels}')
RL_active_result_n_epochs = np.shape(RL_active_result.epochs_df)[0]
RL_z_variable_shapes = [np.shape(x) for x in (RL_active_result.long_z_score, RL_active_result.short_z_score, RL_active_result.long_short_z_score_diff)]
print(f'RL_z_variable_shapes: {RL_z_variable_shapes}')
assert np.all([RL_active_result_n_epochs == s[0] for s in RL_z_variable_shapes]), f"z-variable shapes should match the number of epochs"

## LR and RL epoch labels must be equal to each other!
assert (np.array_equal(rank_order_results.LR_ripple.epochs_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy())), f"Epoch labels must be equal"
new_result_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
new_result_epochs_df.reset_index(drop=True)
new_result_df = pd.DataFrame({
'label': deepcopy(RL_active_result.epochs_df.label.to_numpy()),
'LR_Long_Z': rank_order_results.LR_ripple.long_z_score, 
'LR_Short_Z': rank_order_results.LR_ripple.short_z_score,
'LR_LongShort_Zdiff': rank_order_results.LR_ripple.long_short_z_score_diff,
'RL_Long_Z': rank_order_results.RL_ripple.long_z_score, 
'RL_Short_Z': rank_order_results.RL_ripple.short_z_score,
'RL_LongShort_Zdiff': rank_order_results.RL_ripple.long_short_z_score_diff,
 }, index=new_result_epochs_df.index).reset_index(drop=True)

# new_result_epochs_df.update(new_result_df, errors='raise')
# assert np.shape(new_result_epochs_df)[0] == np.shape(rank_order_results.LR_ripple.epochs_df)[0], f"number of rows must not have changed"
# new_result_epochs_df
new_result_df

In [ ]:
interesting_labels = [4, 148]

In [ ]:
new_result_df[new_result_df.label == 4]

In [ ]:
new_result_df[new_result_df['LR_Long_Z'].abs() > 1.97]


In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 4]

# 2023-12-19 - Transmition Matricies Version

In [ ]:
from PendingNotebookCode import TransitionMatrixComputations

In [ ]:
# pf1D = deepcopy(global_pf1D)
# pf1D = deepcopy(short_pf1D)
# pf1D = deepcopy(long_pf1D)


decoders_dict = track_templates.get_decoders_dict()
binned_x_transition_matrix_higher_order_list_dict = {}

for a_decoder_name, a_decoder in decoders_dict.items():
	a_pf1D = deepcopy(a_decoder.pf)
	binned_x_transition_matrix_higher_order_list_dict[a_decoder_name] = TransitionMatrixComputations._compute_position_transition_matrix(a_pf1D.xbin_labels, a_pf1D.filtered_pos_df['binned_x'].to_numpy())

binned_x_transition_matrix_higher_order_list_dict

In [ ]:
# a_decoder.dec

long_LR_pf1D_Decoder.active_time_window_centers

In [ ]:
# Visualization ______________________________________________________________________________________________________ #
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[2], a_pf1D.xbin_labels, a_pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)



In [ ]:
from scipy.sparse import csr_matrix

rank_order_results.LR_laps.spikes_df

In [ ]:
def _time_transition_matrix(df, bin_size=0.25): # "0.25S"
	# df["bin"] = pd.cut(df["t_rel_seconds"], bins=pd.timedelta_range(df["t_rel_seconds"].min(), df["t_rel_seconds"].max(), freq=bin_size))
	# df["bin"] = pd.cut(
	# 	pd.to_timedelta(df["t_rel_seconds"], unit="s"),
	# 	bins=pd.timedelta_range(df["t_rel_seconds"].min(), df["t_rel_seconds"].max(), freq=bin_size),
	# )
	def get_bin(seconds):
		bin_number = int(seconds // bin_size)
		return bin_number

	df["bin"] = df["t_rel_seconds"].apply(get_bin)

	transition_matrix = csr_matrix((
		counts, (i, j)
	) for i, (bin, group) in df.groupby(["bin", "aclu"]).agg(counts=("aclu", "count")).iterrows()
		for j, counts in group.iteritems()
	)
	transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)

	return transition_matrix


test_spikes_df = deepcopy(rank_order_results.LR_laps.spikes_df)

test_spikes_df = test_spikes_df.spikes.add_binned_time_column()

# [['t_rel_seconds', 'aclu', 'flat_spike_idx', 'Probe_Epoch_id']]]


test_spikes_df


In [ ]:
transition_matrix = _time_transition_matrix(df=test_spikes_df) #  bin_size="0.25S"
transition_matrix

In [ ]:
long_LR_pf1D_Decoder.time_window_centers
# long_LR_pf1D_Decoder.decode_specific_epochs
long_LR_pf1D_Decoder.p_x_given_n

# time_bin_size = 0.25
time_bin_size = 0.1
manual_time_bin_start_t = curr_active_pipeline.sess.t_start
manual_time_bin_end_t = curr_active_pipeline.sess.t_stop

print(f'time_bin_size: {time_bin_size}, manual_time_bins: (start_t: {manual_time_bin_start_t}, end_t: {manual_time_bin_end_t})')
manual_time_window_edges, manual_time_window_edges_binning_info = compute_spanning_bins(None, bin_size=time_bin_size, variable_start_value=manual_time_bin_start_t, variable_end_value=manual_time_bin_end_t) # np.shape(out_digitized_variable_bins)[0] == np.shape(spikes_df)[0]
# debug_print_1D_bin_infos(manual_time_window_edges, 'manual_time_window_edges')

## Build the new decoder with custom params:
new_decoder_pf_params = deepcopy(active_computation_config.pf_params) # should be a PlacefieldComputationParameters
# override some settings before computation:
new_decoder_pf_params.time_bin_size = time_bin_size

## 1D Decoder
new_decoder_pf1D = active_pf_1D
new_1D_decoder_spikes_df = new_decoder_pf1D.filtered_spikes_df.copy()
new_1D_decoder_spikes_df = new_1D_decoder_spikes_df.spikes.add_binned_time_column(manual_time_window_edges, manual_time_window_edges_binning_info, debug_print=False)
new_1D_decoder = BayesianPlacemapPositionDecoder(new_decoder_pf_params.time_bin_size, new_decoder_pf1D, new_1D_decoder_spikes_df, manual_time_window_edges=manual_time_window_edges, manual_time_window_edges_binning_info=manual_time_window_edges_binning_info, debug_print=False)
new_1D_decoder.compute_all() #  --> n = self.


In [ ]:
long_LR_pf1D_Decoder.unit_specific_time_binned_spike_counts.shape

In [ ]:
df

## Matplotlib Layout for Pf1Ds

# 2023-12-19 - Builds a 4x1 Pf1D Placefields figure showing each of the two directions (LR/RL) for Long/Short.

In [ ]:
from neuropy.plotting.ratemaps import enumTuningMap2DPlotVariables

decoders_dict = track_templates.get_decoders_dict()
decoders_dict_keys = list(decoders_dict.keys())
decoder_context_dict = dict(zip(decoders_dict_keys, (long_LR_context, long_RL_context, short_LR_context, short_RL_context)))

print(f'decoders_dict_keys: {decoders_dict_keys}')
plot_kwargs = {}
mosaic = [
        ["ax_pf_tuning_curve"],
        ["ax_pf_occupancy"],
    ]
fig = plt.figure(layout="constrained")
subfigures_dict = dict(zip(list(decoders_dict.keys()), fig.subfigures(nrows=1, ncols=4)))
for a_name, a_subfigure in subfigures_dict.items():
	axd = a_subfigure.subplot_mosaic(mosaic, sharex=True, height_ratios=[8, 1], gridspec_kw=dict(wspace=0, hspace=0.15))
	a_decoder = decoders_dict[a_name]
	active_context = decoder_context_dict[a_name]
	active_display_ctx = active_context.adding_context('display_fn', display_fn_name='plot_ratemaps_1D')
	# active_display_fn_identifying_ctx = curr_active_pipeline.build_display_context_for_filtered_session(filtered_session_name=a_name, display_fn_name='plot_directional_pf1Ds')
	# active_display_fn_identifying_ctx
	ax_pf_1D = a_decoder.pf.plot_ratemaps_1D(ax=axd["ax_pf_tuning_curve"], active_context=active_display_ctx)
	active_display_ctx = active_context.adding_context('display_fn', display_fn_name='plot_occupancy_1D')
	# active_display_ctx_string = active_display_ctx.get_description(separator='|')
	
	display_outputs = a_decoder.pf.plot_occupancy(fig=a_subfigure, ax=axd["ax_pf_occupancy"], active_context=active_display_ctx, **({} | plot_kwargs))
	
	# plot_variable_name = ({'plot_variable': None} | kwargs)
	plot_variable_name = plot_kwargs.get('plot_variable', enumTuningMap2DPlotVariables.OCCUPANCY).name
	active_display_ctx = active_display_ctx.adding_context(None, plot_variable=plot_variable_name)
        
    


# axd = right.subplot_mosaic(mosaic)
# identify_axes(axd)

In [ ]:
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D)


In [ ]:
ax_pf_1D = pf1D.plot_ratemaps_1D(active_context=active_display_fn_identifying_ctx, brev_mode=kwargs.pop('brev_mode', PlotStringBrevityModeEnum.MINIMAL), **kwargs)
active_figure = plt.gcf()

## Setup the plot title and add the session information:
session_identifier = computation_result.sess.get_description() # 'sess_bapun_RatN_Day4_2019-10-15_11-30-06'
fig_label = f'1d_placefields | {session_identifier} | {active_figure.number}'
# print(f'fig_label: {fig_label}')
active_figure.set_label(fig_label)
active_figure.canvas.manager.set_window_title(fig_label) # sets the window's title

In [ ]:

def _display_long_short_pf1D_comparison(owning_pipeline_reference, global_computation_results, computation_results, active_configs, include_includelist=None, save_figure=True, included_any_context_neuron_ids=None, **kwargs):
		""" Displays a figure for comparing the 1D placefields across-epochs (between the short and long tracks). By default renders the second track's placefield flipped over the x-axis and hatched. 
			Usage:

				%matplotlib qt
				active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

				graphics_output_dict = curr_active_pipeline.display('_display_long_short_pf1D_comparison', active_identifying_session_ctx)
				fig, axs, plot_data = graphics_output_dict['fig'], graphics_output_dict['axs'], graphics_output_dict['plot_data']
				

		"""
		reuse_axs_tuple = kwargs.pop('reuse_axs_tuple', None)
		# reuse_axs_tuple = None # plot fresh
		# reuse_axs_tuple=(ax_long_pf_1D, ax_short_pf_1D)
		# reuse_axs_tuple=(ax_long_pf_1D, ax_long_pf_1D) # plot only on long axis
		single_figure = kwargs.pop('single_figure', True)
		debug_print = kwargs.pop('debug_print', False)

		active_config_name = kwargs.pop('active_config_name', None)
		active_context = kwargs.pop('active_context', owning_pipeline_reference.sess.get_context())

		fignum = kwargs.pop('fignum', None)
		if fignum is not None:
			print(f'WARNING: fignum will be ignored but it was specified as fignum="{fignum}"!')
		

		defer_render = kwargs.pop('defer_render', False) 


		# Plot 1D Keywoard args:
		shared_kwargs = kwargs.pop('shared_kwargs', {})
		long_kwargs = kwargs.pop('long_kwargs', {})
		short_kwargs = kwargs.pop('short_kwargs', {})

		shared_kwargs['active_context'] = active_context

		# Allow overriding the sortby for both long and short if a top-level sortby kwarg is passed:
		override_sortby = kwargs.pop('sortby', None)
		if (override_sortby is not None and isinstance(override_sortby, str)):
			print(f'override_sortby: {override_sortby} is being used.')
			# long_sortby = long_kwargs.get('sortby', None)
			# short_sortby = short_kwargs.get('sortby', None)
			# assert (not (long_sortby is not None and isinstance(long_sortby, str)) and (short_sortby is not None and isinstance(short_sortby, str))) # assert no valid sortby values are set for the individual kwargs
			long_kwargs['sortby'] = override_sortby
			short_kwargs['sortby'] = override_sortby

		if include_includelist is None:
			include_includelist = owning_pipeline_reference.active_completed_computation_result_names # ['maze', 'sprinkle']

		long_epoch_name = include_includelist[0] # 'maze1_PYR'
		short_epoch_name = include_includelist[1] # 'maze2_PYR'
		assert len(include_includelist) > 2
		global_epoch_name = include_includelist[-1] # 'maze_PYR'
		if debug_print:
			print(f'include_includelist: {include_includelist}\nlong_epoch_name: {long_epoch_name}, short_epoch_name: {short_epoch_name}, global_epoch_name: {global_epoch_name}')           

		long_results = computation_results[long_epoch_name]['computed_data']
		short_results = computation_results[short_epoch_name]['computed_data']

		# curr_any_context_neurons = _find_any_context_neurons(*[owning_pipeline_reference.computation_results[k].computed_data.pf1D.ratemap.neuron_ids for k in [long_epoch_name, short_epoch_name]])
		curr_any_context_neurons = _find_any_context_neurons(*[a_result.pf1D.ratemap.neuron_ids for a_result in [long_results, short_results]])

		if included_any_context_neuron_ids is None:
			included_any_context_neuron_ids = curr_any_context_neurons
		else:
			# include only the specified neuron_ids:
			included_any_context_neuron_ids = curr_any_context_neurons[np.isin(curr_any_context_neurons, included_any_context_neuron_ids)]

		# SORT BY LONG PEAK LOCATION: Determine the sort indicies to align the placefields by the position on the long:
		### to use this mode, you must pass the string 'peak_long' for both long_kwargs['sortby'] and short_kwargs['sortby']
		long_sortby = long_kwargs.get('sortby', None)
		short_sortby = short_kwargs.get('sortby', None)
		if (long_sortby is not None and isinstance(long_sortby, str)) and (short_sortby is not None and isinstance(short_sortby, str)):
			assert long_sortby == 'peak_long'
			assert short_sortby == 'peak_long'
			new_all_aclus_sort_indicies = determine_long_short_pf1D_indicies_sort_by_peak(owning_pipeline_reference, included_any_context_neuron_ids, debug_print=False)
			# shared_kwargs['sortby'] = (shared_kwargs.get('sortby', None) or new_all_aclus_sort_indicies)
			long_kwargs['sortby'] = new_all_aclus_sort_indicies # (long_kwargs.get('sortby', None) or 
			short_kwargs['sortby'] = new_all_aclus_sort_indicies # (short_kwargs.get('sortby', None) or 
			print(f'DEBUG: new_all_aclus_sort_indicies: {new_all_aclus_sort_indicies}')
		
		(fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, included_any_context_neuron_ids, reuse_axs_tuple=reuse_axs_tuple, single_figure=single_figure,
			shared_kwargs=shared_kwargs, long_kwargs=long_kwargs, short_kwargs=short_kwargs, debug_print=debug_print, **kwargs)

		if single_figure:
			final_context = active_context.adding_context('display_fn', display_fn_name='display_short_long_pf1D_comparison')
		else:
			base_final_context = active_context.adding_context('display_fn', display_fn_name='display_short_long_pf1D_comparison')
			final_context = (base_final_context.overwriting_context(track='long'), base_final_context.overwriting_context(track='short'),) # final context is a tuple of contexts

		def _perform_write_to_file_callback():
			## 2023-05-31 - Reference Output of matplotlib figure to file, along with building appropriate context.
			if single_figure:
				return owning_pipeline_reference.output_figure(final_context, fig_short_pf_1D)
			else:
				owning_pipeline_reference.output_figure(final_context[0], fig_long_pf_1D)
				owning_pipeline_reference.output_figure(final_context[1], fig_short_pf_1D)

		if save_figure:
			active_out_figure_paths = _perform_write_to_file_callback()
		else:
			active_out_figure_paths = []

		graphics_output_dict = MatplotlibRenderPlots(name='display_short_long_pf1D_comparison', figures=(fig_long_pf_1D, fig_short_pf_1D), axes=(ax_long_pf_1D, ax_short_pf_1D), plot_data={}, context=final_context, saved_figures=active_out_figure_paths)
		graphics_output_dict['plot_data'] = {'included_any_context_neuron_ids': included_any_context_neuron_ids, 'sort_indicies': (long_sort_ind, short_sort_ind), 'colors':(long_neurons_colors_array, short_neurons_colors_array)}

		return graphics_output_dict
